In [2]:
import gc
import os
import os.path
import time
import random

import PIL.Image as Image
import matplotlib.pyplot as plt
import numpy as np
from six.moves import cPickle as pickle
from skimage.transform import resize
from skimage.transform import rotate
from scipy.ndimage import gaussian_filter


class MultiDigitSVHNData(object):
    def __init__(self, digit_count=5, input_total_data_count=1, 
                image_size=32, use_standard_score=False, add_margin=False, margin_ratio=0.1,
                random_position_count=0, random_rotate_count=0, apply_gaussian_filter=False
                ):
        random.seed(42)
        self.use_standard_score = use_standard_score
        self.add_margin = add_margin
        self.margin_ratio = margin_ratio
        self.random_position_count = random_position_count
        self.random_rotate_count = random_rotate_count
        self.apply_gaussian_filter = apply_gaussian_filter
        self.digit_count = digit_count
        self.image_width = image_size
        self.image_height = image_size
        self.target_digit_num_labels = 10
        self.num_channels = 3

        self.input_total_data_count = input_total_data_count
        self.total_data_count = input_total_data_count  # ds.num_examples
        self.update_data_counts()

        self.train_data = None
        self.train_label_length = None
        self.train_label_digits = None
        self.validation_data = None
        self.validation_label_length = None
        self.validation_label_digits = None
        self.test_data = None
        self.test_label_length = None
        self.test_label_digits = None

    def update_data_counts(self):
        self.train_data_count = int(0.9 * self.total_data_count)
        self.validation_data_count = int(0.05 * self.total_data_count)
        self.test_data_count = self.total_data_count - self.train_data_count - self.validation_data_count

    def print_image(self, image):
        sample_image = image
        plt.figure()
        plt.imshow(sample_image)  # display it
        gc.collect()

    @staticmethod
    def maybe_pickle(digit_count, total_data_count, image_size=54, force=False, 
                     use_standard_score=False, add_margin=False, margin_ratio=0.1,
                     random_position_count=0, random_rotate_count=0, 
                     apply_gaussian_filter=False):
        filename = 'svhn/reader/svhn_' + str(digit_count) \
            + '_' + str(image_size) \
            + '_' + str(total_data_count) \
            + '_' + str(use_standard_score) \
            + '_' + str(add_margin) \
            + '_' + str(margin_ratio) \
            + '_' + str(apply_gaussian_filter) \
        
        if random_position_count > 0:
            filename += '_position' + str(random_position_count) \
        
        if random_rotate_count > 0:
            filename += '_rotate' + str(random_rotate_count) \
            
        filename += '.pickle'

        if os.path.exists(filename) and not force:
            # You may override by setting force=True.
            print('%s already present - Skipping pickling.' % filename)
            svhn_data = MultiDigitSVHNData.load_pickle(filename)
        else:
            print('Pickling %s.' % filename)
            svhn_data = MultiDigitSVHNData(digit_count=digit_count, input_total_data_count=total_data_count,
                                            image_size=image_size, use_standard_score=use_standard_score, add_margin=add_margin,
                                            random_position_count=random_position_count, random_rotate_count=random_rotate_count,
                                            margin_ratio=margin_ratio, apply_gaussian_filter=apply_gaussian_filter)
            svhn_data.load_data()
            try:
                MultiDigitSVHNData.write_pickle(filename, svhn_data)
            except Exception as e:
                print('Unable to save data to', filename, ':', e)
        return svhn_data

    @staticmethod
    def write_pickle(filename, data):
        with open(filename, 'wb') as f:
            pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)

    @staticmethod
    def load_pickle(filename):
        with open(filename, 'rb') as f:
            return pickle.load(f)

    def reformat_target_length(self, target_length):
        return (np.arange(self.digit_count + 1) == target_length).astype(np.float32)

    def reformat_target_digits(self, target_digits):
        return (np.arange(10) == target_digits).astype(np.float32)
    
    def generate_data(self, rows, total_data_index, target_image_length, data_index, target_image,top,bottom,left,right,rotate_rand=False):
        self.total_label[total_data_index][0] = target_image_length

        for digit_index in range(0, target_image_length):
            self.total_label[total_data_index][1 + digit_index] = rows[1][data_index][digit_index][0]
            
        rand_rotate_angle = 0
        if rotate_rand:
            if random.choice([True, False]):
                rand_rotate_angle = random.randrange(1,91)
            else:
                rand_rotate_angle = random.randrange(-91, -1)

        for channel_index in range(0, self.num_channels):
            if self.use_standard_score:
                mean = np.mean(target_image[:][:][channel_index], dtype='float32')
                std = np.std(target_image[:][:][channel_index], dtype='float32', ddof=1)
                if std < 1e-4:std = 1
                target_image[:][:][channel_index] = ((target_image[:][:][channel_index] - mean) / std).astype('uint8')
            
            croped = target_image[channel_index][top:bottom, left:right]
            if rotate_rand:
                croped = rotate(croped, rand_rotate_angle)
            resized_image = resize(croped,output_shape=[self.image_width, self.image_height])
            for image_height_index in range(0, self.image_height):
                for image_width_index in range(0, self.image_width):
                    self.total_data[total_data_index][image_height_index][image_width_index][channel_index] = \
                        resized_image[image_height_index][image_width_index] * 255

        if self.apply_gaussian_filter:
            self.total_data[total_data_index] = gaussian_filter(self.total_data[total_data_index], 1)
        total_data_index += 1
        
        return total_data_index

    def load_data(self):
        from fuel.config_parser import config
        from fuel.datasets.svhn import SVHN
        config.data_path = './svhn/converted'
        ds = SVHN(which_format=1, which_sets=('train', 'test', 'extra'))
        fetched_total_data_count = min(ds.num_examples, self.input_total_data_count)
        rows = np.array(ds.get_data(state=ds.open(), request=range(0, fetched_total_data_count)))
        rows_len = len(rows)
        print("rows[5][0]", rows[5][0].shape, rows[5][0].dtype)

        new_cnt = 0
        for data_index in range(0, fetched_total_data_count):
            target_image = rows[5][data_index]
            target_image_length = len(rows[1][data_index])
            if target_image_length <= self.digit_count:
                new_cnt += 1
                if self.add_margin:
                    if self.random_position_count > 0:
                        new_cnt += self.random_position_count
                    if self.random_rotate_count > 0:
                        new_cnt += self.random_rotate_count
        self.total_data_count = new_cnt
        self.update_data_counts()

        self.total_data = np.zeros((self.total_data_count, self.image_height, self.image_width, self.num_channels), dtype='uint8')
        self.total_label = np.zeros((self.total_data_count, self.digit_count + 1), dtype=int)
        print("total_data", self.total_data.shape)
        print("total_label", self.total_label.shape)

        print("start : {}".format(time.strftime("%Y-%m-%dT%H:%M:%S%z")))
        total_data_index = 0
        for data_index in range(0, fetched_total_data_count):
            target_image = rows[5][data_index]
            target_image_length = len(rows[1][data_index])
            top = int(np.min(rows[3][data_index]))
            bottom = int(np.max(rows[0][data_index] + rows[3][data_index]))
            left = int(np.min(rows[2][data_index]))
            right = int(np.max(rows[2][data_index] + rows[4][data_index]))
            
            rand_max_width = 0
            rand_max_height = 0

            if self.add_margin:
                width = right - left
                height = bottom - top
                top = max(0, int(top - height * self.margin_ratio))
                bottom = min(len(target_image[0]), int(bottom + height * self.margin_ratio))
                left = max(0, int(left - width * self.margin_ratio))
                right = min(len(target_image[0][0]), int(right + width * self.margin_ratio))
                rand_max_width = int(width * self.margin_ratio)
                rand_max_height = int(height * self.margin_ratio)

            if target_image_length <= self.digit_count:
                #def generate_data(self, total_data_index, target_image_length, data_index, target_image,top,bottom,left,right):
                total_data_index = self.generate_data(rows,total_data_index,target_image_length,data_index,target_image,top,bottom,left,right)
                if self.add_margin:
                    if self.random_position_count > 0:
                        for rand_i in range(0, self.random_position_count):
                            rand_width = random.randrange(-rand_max_width, rand_max_width)
                            rand_height = random.randrange(-rand_max_height, rand_max_height)
                            rand_top = max(0, top + rand_height)
                            rand_bottom = min(len(target_image[0]), bottom + rand_height)
                            rand_left = max(0, left + rand_width)
                            rand_right = min(len(target_image[0][0]), right + rand_width)
                            
                            total_data_index = self.generate_data(rows,total_data_index,target_image_length,data_index,target_image,
                                                                  rand_top,rand_bottom,rand_left,rand_right)
                    if self.random_rotate_count > 0:
                        for rand_i in range(0, self.random_rotate_count):
                            total_data_index = self.generate_data(rows,total_data_index,target_image_length,data_index,target_image
                                                                  ,top,bottom,left,right, rotate_rand=True)
                    
        print("end : {}".format(time.strftime("%Y-%m-%dT%H:%M:%S%z")))

        self.train_data = self.total_data[:self.train_data_count]
        self.train_label = self.total_label[:self.train_data_count]
        print("train_data", self.train_data.shape)
        print("train_label", self.train_label.shape)
        
        self.validation_data = self.total_data[self.train_data_count:self.train_data_count + self.validation_data_count]
        self.validation_label = self.total_label[
                                       self.train_data_count:self.train_data_count + self.validation_data_count]
        print("validation_data", self.validation_data.shape)
        print("validation_label", self.validation_label.shape)

        self.test_data = self.total_data[self.train_data_count + self.validation_data_count:]
        self.test_label = self.total_label[self.train_data_count + self.validation_data_count:]
        print("test_data", self.test_data.shape)
        print("test_label", self.test_label.shape)



In [3]:
import re

import numpy as np
import tensorflow as tf

from tensorflow.python.framework import graph_util
from tensorflow.python.framework import tensor_shape
from tensorflow.python.platform import gfile
from tensorflow.python.util import compat



class MultiDigitModel(object):
    def __init__(self, multi_digit_dataset, digit_count=1, num_channels=1, pooling_stride=2, num_steps=10001,
                 batch_size=128, patch_size=5, num_convs=None, num_fc_1=1600, num_fc_2=512, beta=0.001,
                 use_drop_out=False, drop_out_rate=0.5, use_norm=False, use_LCN=False,
                 learning_rate_start=0.1, learning_rate_decay_rate=0.1, add_l2_loss=False):
        if num_convs is None:
            num_convs = [24, 32, 32, 32, 32, 32, 32, 64]

        self.num_channels = num_channels
        self.pooling_stride = pooling_stride

        self.num_steps = num_steps
        self.batch_size = batch_size
        self.patch_size = patch_size
        self.num_conv_1 = num_convs[0]
        self.num_conv_2 = num_convs[1]
        self.num_conv_3 = num_convs[2]
        self.num_conv_4 = num_convs[3]
        self.num_conv_5 = num_convs[4]
        self.num_conv_6 = num_convs[5]
        self.num_conv_7 = num_convs[6]
        self.num_conv_8 = num_convs[7]

        self.last_num_conv = 0
        for i in range(7, -1, -1):
            if num_convs[i] > 0:
                self.last_num_conv = num_convs[i]
                break

        self.num_fc_1 = num_fc_1
        self.num_fc_2 = num_fc_2

        self.beta = beta
        self.learning_rate_start = learning_rate_start
        self.learning_rate_decay_rate = learning_rate_decay_rate

        self.digit_count = digit_count
        self.multi_digit_dataset = multi_digit_dataset
        self.graph = tf.Graph()

        self.add_l2_loss = add_l2_loss
        self.NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN = 50000
        self.NUM_EXAMPLES_PER_EPOCH_FOR_EVAL = 10000

        # Constants describing the training process.
        self.MOVING_AVERAGE_DECAY = 0.9999     # The decay to use for the moving average.
        self.NUM_EPOCHS_PER_DECAY = 350.0      # Epochs after which learning rate decays.
        
        self.use_norm = use_norm
        self.use_drop_out = use_drop_out
        self.drop_out_rate = drop_out_rate
        self.use_LCN = use_LCN


    @staticmethod
    def accuracy_digit(p_length, p_digits,
                       batch_length_labels, batch_digits_labels, digit):
        eq_count = 0.0
        total_count = 0.0
        for i in range(0, len(p_digits[digit])):
            if np.argmax(batch_length_labels[i]) >= digit:
                total_count += 1.0
                if np.argmax(p_digits[digit][i]) == np.argmax(batch_digits_labels[i][digit]):
                    eq_count += 1.0
                    #         elif digit == 1:
                    #           print("np.argmax(p_digits[digit][i]):{}, np.argmax(batch_digits_labels[i][digit]:{}".format(np.argmax(p_digits[digit][i]), np.argmax(batch_digits_labels[i][digit])))

        if total_count == 0:
            return 0
        return eq_count / total_count * 100

    @staticmethod
    def accuracy_length(p_length, p_0, p_1, p_2, p_3, p_4,
                        batch_length_labels, batch_digits_labels):
        eq_count = 0.0
        for i in range(0, len(p_length)):
            if np.argmax(p_length[i]) == np.argmax(batch_length_labels[i]):
                eq_count += 1.0
        return eq_count / len(p_length) * 100

    @staticmethod
    def accuracy(p_length, p_digits,
                 batch_length_labels, batch_digits_labels):
        eq_count = 0.0
        for row_index in range(0, len(p_length)):
            # print("row_index:{}".format(row_index))
            one_based_length_predicted = np.argmax(p_length[row_index])
            one_based_length_real = np.argmax(batch_length_labels[row_index])

            # print("one_based_length_predicted : {}, one_based_length_real :{}".format(one_based_length_predicted, one_based_length_real))
            if one_based_length_predicted == one_based_length_real:
                is_equal = True
                for digit_index in range(0, one_based_length_real):
                    # print("\tdigit_index:{}".format(digit_index))
                    if np.argmax(p_digits[digit_index][row_index]) != np.argmax(
                            batch_digits_labels[row_index][digit_index]):
                        # print("\t\tnp.argmax(p_digits[digit_index][row_index]) :{}, np.argmax(batch_digits_labels[row_index][digit_index]) :{}".format(np.argmax(p_digits[digit_index][row_index]), np.argmax(batch_digits_labels[row_index][digit_index])))
                        is_equal = False
                        break
                if is_equal:
                    eq_count += 1.0

        return eq_count / len(p_length) * 100, eq_count, len(p_length)
    
    
    def _activation_summary(self, x):
        tf.histogram_summary(x.op.name + '/activations', x)
        tf.scalar_summary(x.op.name + '/sparsity', tf.nn.zero_fraction(x))

    def _add_loss_summaries(self, total_loss):
        # Compute the moving average of all individual losses and the total loss.
        loss_averages = tf.train.ExponentialMovingAverage(0.9, name='avg')
        losses = tf.get_collection('losses')
        loss_averages_op = loss_averages.apply(losses + [total_loss])

        # Attach a scalar summary to all individual losses and the total loss; do the
        # same for the averaged version of the losses.
        for l in losses + [total_loss]:
        # Name each loss as '(raw)' and name the moving average version of the loss
        # as the original loss name.
            tf.scalar_summary(l.op.name +' (raw)', l)
            tf.scalar_summary(l.op.name, loss_averages.average(l))

        return loss_averages_op

        
    def _variable_on_cpu(self, name, shape, initializer):
        return tf.get_variable(name, shape, initializer=initializer, dtype=tf.float32)
        
        
    def _variable_with_weight_decay(self, name, shape, stddev, wd):
        var = self._variable_on_cpu(
            name,
            shape,
            tf.truncated_normal_initializer(stddev=stddev, dtype=tf.float32))
        if wd is not None:
            weight_decay = tf.mul(tf.nn.l2_loss(var), wd, name='weight_loss')
            tf.add_to_collection('losses', weight_decay)
        return var
    
    def conv_layer(self, number, input_data, input_num, output_num):
        with tf.variable_scope('conv_' + str(number)) as scope:
            kernel = self._variable_with_weight_decay('weights',
                             shape=[self.patch_size, self.patch_size, input_num, output_num],
                             stddev=5e-2,
                             wd=0.0)
            conv = tf.nn.conv2d(input_data, kernel, [1, 1, 1, 1], padding='SAME')
            biases = self._variable_on_cpu('biases', [output_num], tf.constant_initializer(0.0))
            pre_activation = tf.nn.bias_add(conv, biases)
            conv = tf.nn.relu(pre_activation, name=scope.name)
            self._activation_summary(conv)
        if self.use_norm:
            conv = tf.nn.lrn(conv, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm' + str(number))
        return conv
    
    # inference.
    def inference(self, data, is_training=True):
        stride = 2
        conv = None
        with tf.variable_scope('conv_1') as scope:
            kernel = self._variable_with_weight_decay('weights',
                             shape=[self.patch_size, self.patch_size, self.num_channels, self.num_conv_1],
                             stddev=5e-2,
                             wd=0.0)
            conv = tf.nn.conv2d(data, kernel, [1, 1, 1, 1], padding='SAME')
            biases = self._variable_on_cpu('biases', [self.num_conv_1], tf.constant_initializer(0.0))
            pre_activation = tf.nn.bias_add(conv, biases)
            conv1 = tf.nn.relu(pre_activation, name=scope.name)
            self._activation_summary(conv1)
        pool1 = tf.nn.max_pool(conv1, [1, 2, 2, 1], [1, stride, stride, 1], padding='SAME', name='pool1')
        if self.use_norm:
            conv = tf.nn.lrn(pool1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm1')
        else:
            conv = pool1

        with tf.variable_scope('conv_2') as scope:
            kernel = self._variable_with_weight_decay('weights',
                             shape=[self.patch_size, self.patch_size, self.num_conv_1, self.num_conv_2],
                             stddev=5e-2,
                             wd=0.0)
            conv = tf.nn.conv2d(conv, kernel, [1, 1, 1, 1], padding='SAME')
            biases = self._variable_on_cpu('biases', [self.num_conv_2], tf.constant_initializer(0.0))
            pre_activation = tf.nn.bias_add(conv, biases)
            conv2 = tf.nn.relu(pre_activation, name=scope.name)
            self._activation_summary(conv2)
        if self.use_norm:
            conv2 = tf.nn.lrn(conv2, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm2')
        pool2 = tf.nn.max_pool(conv2, [1, 2, 2, 1], [1, stride, stride, 1], padding='SAME', name='pool2')
        conv = pool2
        
        if self.num_conv_3 > 0:
            conv = self.conv_layer(3, conv, self.num_conv_2, self.num_conv_3)
        if self.num_conv_4 > 0:
            conv = self.conv_layer(4, conv, self.num_conv_3, self.num_conv_4)
        if self.num_conv_5 > 0:
            conv = self.conv_layer(5, conv, self.num_conv_4, self.num_conv_5)
        if self.num_conv_6 > 0:
            conv = self.conv_layer(6, conv, self.num_conv_5, self.num_conv_6)
        if self.num_conv_7 > 0:
            conv = self.conv_layer(7, conv, self.num_conv_6, self.num_conv_7)
        if self.num_conv_8 > 0:
            conv = self.conv_layer(8, conv, self.num_conv_7, self.num_conv_8)
        
        if self.use_drop_out:
            conv = tf.nn.dropout(conv, self.drop_out_rate)
        
        with tf.variable_scope('local_1') as scope:
            shape = conv.get_shape().as_list()
            reshape = tf.reshape(conv, [shape[0], -1])
            dim = reshape.get_shape()[1].value
            weights = self._variable_with_weight_decay('weights', shape=[dim, self.num_fc_1],stddev=0.04, wd=0.004)
            biases = self._variable_on_cpu('biases', [self.num_fc_1], tf.constant_initializer(0.1))
            local3 = tf.nn.relu(tf.matmul(reshape, weights) + biases, name=scope.name)
            self._activation_summary(local3)

        with tf.variable_scope('local_2') as scope:
            weights = self._variable_with_weight_decay('weights', shape=[self.num_fc_1, self.num_fc_2],stddev=0.04, wd=0.004)
            biases = self._variable_on_cpu('biases', [self.num_fc_2], tf.constant_initializer(0.1))
            local4 = tf.nn.relu(tf.matmul(local3, weights) + biases, name=scope.name)
            self._activation_summary(local4)

        with tf.variable_scope('digit_length') as scope:
            weights = self._variable_with_weight_decay('weights', [self.num_fc_2, self.multi_digit_dataset.digit_count + 1],stddev=1.0/self.num_fc_2, wd=0.0)
            biases = self._variable_on_cpu('biases', [self.multi_digit_dataset.digit_count + 1],tf.constant_initializer(0.0))
            digit_length_logit = tf.add(tf.matmul(local4, weights), biases, name=scope.name)
            self._activation_summary(digit_length_logit)

        with tf.variable_scope('digit_0') as scope:
            weights = self._variable_with_weight_decay('weights', [self.num_fc_2, 10],stddev=1.0/self.num_fc_2, wd=0.0)
            biases = self._variable_on_cpu('biases', [10],tf.constant_initializer(0.0))
            digits_0_logit = tf.add(tf.matmul(local4, weights), biases, name=scope.name)
            self._activation_summary(digits_0_logit)
        with tf.variable_scope('digit_1') as scope:
            weights = self._variable_with_weight_decay('weights', [self.num_fc_2, 10],stddev=1.0/self.num_fc_2, wd=0.0)
            biases = self._variable_on_cpu('biases', [10],tf.constant_initializer(0.0))
            digits_1_logit = tf.add(tf.matmul(local4, weights), biases, name=scope.name)
            self._activation_summary(digits_1_logit)
        with tf.variable_scope('digit_2') as scope:
            weights = self._variable_with_weight_decay('weights', [self.num_fc_2, 10],stddev=1.0/self.num_fc_2, wd=0.0)
            biases = self._variable_on_cpu('biases', [10],tf.constant_initializer(0.0))
            digits_2_logit = tf.add(tf.matmul(local4, weights), biases, name=scope.name)
            self._activation_summary(digits_2_logit)
        with tf.variable_scope('digit_3') as scope:
            weights = self._variable_with_weight_decay('weights', [self.num_fc_2, 10],stddev=1.0/self.num_fc_2, wd=0.0)
            biases = self._variable_on_cpu('biases', [10],tf.constant_initializer(0.0))
            digits_3_logit = tf.add(tf.matmul(local4, weights), biases, name=scope.name)
            self._activation_summary(digits_3_logit)
        with tf.variable_scope('digit_4') as scope:
            weights = self._variable_with_weight_decay('weights', [self.num_fc_2, 10],stddev=1.0/self.num_fc_2, wd=0.0)
            biases = self._variable_on_cpu('biases', [10],tf.constant_initializer(0.0))
            digits_4_logit = tf.add(tf.matmul(local4, weights), biases, name=scope.name)
            self._activation_summary(digits_4_logit)

        return digit_length_logit, digits_0_logit, digits_1_logit, digits_2_logit, digits_3_logit, digits_4_logit

    
    def loss_(self, logits, length_label, digit_labels):
        # Calculate the average cross entropy loss across the batch.
        length_label = tf.cast(length_label, tf.int64)
        digit_labels = tf.cast(digit_labels, tf.int64)
#         print(logits[0], length_label, digit_labels)
        cross_entropy_sum = tf.nn.softmax_cross_entropy_with_logits(logits[0], length_label, name='cross_entropy_for_length')
        for i in range(0, self.digit_count):
#             print(i, logits[i + 1], digit_labels[:,i])
            cross_entropy_sum += tf.nn.softmax_cross_entropy_with_logits(logits[i + 1], digit_labels[:, i], name='cross_entropy_for_digit' + str(i))
        cross_entropy_mean = tf.reduce_mean(cross_entropy_sum, name='cross_entropy')
        tf.add_to_collection('losses', cross_entropy_mean)

        # The total loss is defined as the cross entropy loss plus all of the weight
        # decay terms (L2 loss).
        return tf.add_n(tf.get_collection('losses'), name='total_loss')

    
    def train(self, total_loss, global_step):
        # Variables that affect learning rate.
        num_batches_per_epoch = self.NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN / self.batch_size
        decay_steps = int(num_batches_per_epoch * self.NUM_EPOCHS_PER_DECAY)

        # Decay the learning rate exponentially based on the number of steps.
        lr = tf.train.exponential_decay(self.learning_rate_start,
                                      global_step,
                                      decay_steps,
                                      self.learning_rate_decay_rate,
                                      staircase=True)
        tf.scalar_summary('learning_rate', lr)

        # Generate moving averages of all losses and associated summaries.
        loss_averages_op = self._add_loss_summaries(total_loss)

        # Compute gradients.
        with tf.control_dependencies([loss_averages_op]):
            #opt = tf.train.GradientDescentOptimizer(lr)
            opt = tf.train.AdagradOptimizer(lr)
            grads = opt.compute_gradients(total_loss)

        # Apply gradients.
        apply_gradient_op = opt.apply_gradients(grads, global_step=global_step)

        # Add histograms for trainable variables.
        for var in tf.trainable_variables():
            tf.histogram_summary(var.op.name, var)

        # Add histograms for gradients.
        for grad, var in grads:
            if grad is not None:
                tf.histogram_summary(var.op.name + '/gradients', grad)

        # Track the moving averages of all trainable variables.
        variable_averages = tf.train.ExponentialMovingAverage(self.MOVING_AVERAGE_DECAY, global_step)
        variables_averages_op = variable_averages.apply(tf.trainable_variables())

        with tf.control_dependencies([apply_gradient_op, variables_averages_op]):
            train_op = tf.no_op(name='train')

        return train_op
    
    def run(self):
        import time
        
        with self.graph.as_default():
            # Input data.
            tf_train_dataset = tf.placeholder(tf.float32, shape=(
                self.batch_size, self.multi_digit_dataset.image_width, self.multi_digit_dataset.image_height,
                self.multi_digit_dataset.num_channels), name="tf_train_dataset")
            print("tf_train_dataset : {}".format(tf_train_dataset))
            tf_train_length_labels = tf.placeholder(tf.float32,
                                                    shape=(self.batch_size, self.multi_digit_dataset.digit_count + 1))
            print("tf_train_length_labels : {}".format(tf_train_length_labels))
            tf_train_digits_labels = tf.placeholder(tf.float32,
                                                    shape=(self.batch_size, self.multi_digit_dataset.digit_count, 10))
            print("tf_train_digits_labels : {}".format(tf_train_digits_labels))

            # Training computation.
            logits = self.inference(tf_train_dataset, True)
            #   digits_1_mult = tf.to_float((tf.argmax(tf.nn.softmax(tf_train_length_labels), dimension=1) > 1))
            #   digits_2_mult = tf.to_float((tf.argmax(tf.nn.softmax(tf_train_length_labels), dimension=1) > 2))
            #   digits_3_mult = tf.to_float((tf.argmax(tf.nn.softmax(tf_train_length_labels), dimension=1) > 3))
            #   digits_4_mult = tf.to_float((tf.argmax(tf.nn.softmax(tf_train_length_labels), dimension=1) > 4))
            loss = self.loss_(logits, tf_train_length_labels, tf_train_digits_labels)

            global_step = tf.Variable(0, trainable=False)
            train_op = self.train(loss, global_step)

            # Predictions for the training, validation, and test data.
            train_length_prediction = tf.nn.softmax(logits[0], name='p_digit_length')
            train_digits_0_prediction = tf.nn.softmax(logits[1], name='p_digit_0')
            train_digits_1_prediction = tf.nn.softmax(logits[2], name='p_digit_1')
            train_digits_2_prediction = tf.nn.softmax(logits[3], name='p_digit_2')
            train_digits_3_prediction = tf.nn.softmax(logits[4], name='p_digit_3')
            train_digits_4_prediction = tf.nn.softmax(logits[5], name='p_digit_4')
            
            summary_op = tf.merge_all_summaries()
            init = tf.initialize_all_variables()
            session = tf.Session()
            session.run(init)
            summary_writer = tf.train.SummaryWriter('logs/tensorboard/voyageth/svhn/' + time.strftime("%Y-%m-%dT%H:%M:%S%z"), session.graph)

            saver = tf.train.Saver()
            
            print('Initialized')
            for step in range(self.num_steps):
                offset = (step * self.batch_size) % (self.multi_digit_dataset.train_data.shape[0] - self.batch_size)
                batch_data = self.multi_digit_dataset.train_data[offset:(offset + self.batch_size), :, :, :]
                batch_length_labels = self.multi_digit_dataset.reformat_target_length(self.multi_digit_dataset.train_label[offset:(offset + self.batch_size), 0, None])
                batch_digits_labels = self.multi_digit_dataset.reformat_target_digits(self.multi_digit_dataset.train_label[offset:(offset + self.batch_size), 1:, None])

                feed_dict = {tf_train_dataset: batch_data, tf_train_length_labels: batch_length_labels,
                             tf_train_digits_labels: batch_digits_labels}

                _, summary_str, l, p_length, p_0, p_1, p_2, p_3, p_4 = session.run([train_op, summary_op, loss, train_length_prediction, train_digits_0_prediction, train_digits_1_prediction,
                     train_digits_2_prediction, train_digits_3_prediction, train_digits_4_prediction],feed_dict=feed_dict)
                if step % 100 == 0:
                    summary_writer.add_summary(summary_str, step)

                if step % 500 == 0:
                    print('Minibatch loss at step %d: %f' % (step, l))
                    accuracy_result = self.accuracy_length(p_length, p_0, p_1, p_2, p_3, p_4, batch_length_labels,
                                                           batch_digits_labels)
                    print('Minibatch accuracy_length: %.1f%%' % accuracy_result)
                    for k in range(0, self.digit_count):
                        accuracy_result = self.accuracy_digit(p_length, [p_0, p_1, p_2, p_3, p_4], batch_length_labels,
                                                              batch_digits_labels, k)
                        print("Minibatch accuracy_digit_{}".format(k) + ": %.1f%%" % accuracy_result)
                    accuracy_result, _, _ = self.accuracy(p_length, [p_0, p_1, p_2, p_3, p_4], batch_length_labels,
                                                    batch_digits_labels)
                    print('Minibatch accuracy: %.1f%%' % accuracy_result)

                    validation_accuracy_numerator = 0
                    validation_accuracy_denominator = 0
                    for step in range(0, len(self.multi_digit_dataset.validation_label) / self.batch_size):
                        valid_offset = (step * self.batch_size) % (self.multi_digit_dataset.validation_data.shape[0] - self.batch_size)
                        valid_batch_data = self.multi_digit_dataset.validation_data[valid_offset:(valid_offset + self.batch_size), :, :, :]
                        valid_batch_length_labels = self.multi_digit_dataset.reformat_target_length(self.multi_digit_dataset.validation_label[valid_offset:(valid_offset + self.batch_size), 0, None])
                        valid_batch_digits_labels = self.multi_digit_dataset.reformat_target_digits(self.multi_digit_dataset.validation_label[valid_offset:(valid_offset + self.batch_size), 1:, None])

                        feed_dict = {tf_train_dataset: valid_batch_data, tf_train_length_labels: valid_batch_length_labels,
                             tf_train_digits_labels: valid_batch_digits_labels}

                        p_length, p_0, p_1, p_2, p_3, p_4 = session.run([train_length_prediction, 
                                                                         train_digits_0_prediction, 
                                                                         train_digits_1_prediction,
                                                                         train_digits_2_prediction, 
                                                                         train_digits_3_prediction, 
                                                                         train_digits_4_prediction],feed_dict=feed_dict)

                        accuracy_result, numerator, denominator = self.accuracy(p_length, [p_0, p_1, p_2, p_3, p_4], 
                                                        valid_batch_length_labels, valid_batch_digits_labels)
                        validation_accuracy_numerator += numerator
                        validation_accuracy_denominator += denominator
                    accuracy_result = validation_accuracy_numerator / validation_accuracy_denominator * 100
                    print('Validation accuracy: %.1f%%' % accuracy_result)
                    print("finish : {}".format(time.strftime("%Y-%m-%dT%H:%M:%S%z")))

            test_accuracy_numerator = 0
            test_accuracy_denominator = 0
            for step in range(0, len(self.multi_digit_dataset.test_label) / self.batch_size):
                test_offset = (step * self.batch_size) % (self.multi_digit_dataset.test_data.shape[0] - self.batch_size)
                test_batch_data = self.multi_digit_dataset.test_data[test_offset:(test_offset + self.batch_size), :, :, :]
                test_batch_length_labels = self.multi_digit_dataset.reformat_target_length(self.multi_digit_dataset.test_label[test_offset:(test_offset + self.batch_size), 0, None])
                test_batch_digits_labels = self.multi_digit_dataset.reformat_target_digits(self.multi_digit_dataset.test_label[test_offset:(test_offset + self.batch_size), 1:, None])

                feed_dict = {tf_train_dataset: test_batch_data, tf_train_length_labels: test_batch_length_labels,
                     tf_train_digits_labels: test_batch_digits_labels}

                p_length, p_0, p_1, p_2, p_3, p_4 = session.run([train_length_prediction, 
                                                                 train_digits_0_prediction, 
                                                                 train_digits_1_prediction,
                                                                 train_digits_2_prediction, 
                                                                 train_digits_3_prediction, 
                                                                 train_digits_4_prediction],feed_dict=feed_dict)

                accuracy_result, numerator, denominator = self.accuracy(p_length, [p_0, p_1, p_2, p_3, p_4], 
                                                test_batch_length_labels, test_batch_digits_labels)
                test_accuracy_numerator += numerator
                test_accuracy_denominator += denominator
            accuracy_result = test_accuracy_numerator / test_accuracy_denominator * 100
            print('Test accuracy: %.1f%%' % accuracy_result)
            finish_time = time.strftime("%Y-%m-%dT%H:%M:%S%z")
            print("finish : {}".format(finish_time))
#             path = saver.save(session, "{}_MultiDigitNN.pb".format(time.strftime("%Y-%m-%dT%H:%M:%S%z")))
#             print("Model saved. path : %s" % path)

            output_graph_def = graph_util.convert_variables_to_constants(
              session, self.graph.as_graph_def(), ['p_digit_length', 'p_digit_0', 'p_digit_1', 'p_digit_2', 'p_digit_3', 'p_digit_4'])
            with gfile.FastGFile('./graph/model_{}.pb'.format(finish_time), 'wb') as f:
                f.write(output_graph_def.SerializeToString())
                        



In [28]:
# Minibatch loss at step 10000: 2.655384
# Minibatch accuracy_length: 96.9%
# Minibatch accuracy_digit_0: 94.5%
# Minibatch accuracy_digit_1: 92.2%
# Minibatch accuracy_digit_2: 87.3%
# Minibatch accuracy_digit_3: 96.2%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 79.7%
# finish : 2016-11-21T17:13:47+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        learning_rate_start=0.005,
                        digit_count=5,
                        num_channels=3,
                        num_steps=10001, 
                        batch_size=128, 
                        num_convs=[16,32,32,32,64,64,64,64], 
                        num_fc_1=384, 
                        num_fc_2=192,
                       ).run()

svhn/reader/svhn_5_32_300000_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(128, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(128, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(128, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 18.446604
Minibatch accuracy_length: 0.0%
Minibatch accuracy_digit_0: 28.1%
Minibatch accuracy_digit_1: 5.5%
Minibatch accuracy_digit_2: 4.4%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 50.0%
Minibatch accuracy: 0.0%
finish : 2016-11-21T17:06:53+0900
Minibatch loss at step 50: 11.049203
Minibatch accuracy_length: 55.5%
Minibatch accuracy_digit_0: 30.5%
Minibatch accuracy_digit_1: 24.2%
Minibatch accuracy_digit_2: 65.8%
Minibatch accuracy_digit_3: 88.4%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 3.9%
finish : 2016-11-21T17:06:56+0900
Minibatch loss at step 100: 11.136753
Minibatch accuracy_length: 

In [33]:
# Minibatch loss at step 10000: 1.572002
# Minibatch accuracy_length: 98.4%
# Minibatch accuracy_digit_0: 92.2%
# Minibatch accuracy_digit_1: 95.3%
# Minibatch accuracy_digit_2: 88.3%
# Minibatch accuracy_digit_3: 93.3%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 79.7%
# finish : 2016-11-21T17:32:39+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=10001, 
                        batch_size=64,  
                        num_convs=[24,32,48,64,64,64,64,64], 
                        num_fc_1=128, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(64, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(64, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(64, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 19.108332
Minibatch accuracy_length: 1.6%
Minibatch accuracy_digit_0: 9.4%
Minibatch accuracy_digit_1: 10.9%
Minibatch accuracy_digit_2: 0.0%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
finish : 2016-11-21T17:20:47+0900
Minibatch loss at step 50: 8.264974
Minibatch accuracy_length: 54.7%
Minibatch accuracy_digit_0: 21.9%
Minibatch accuracy_digit_1: 31.2%
Minibatch accuracy_digit_2: 73.1%
Minibatch accuracy_digit_3: 94.1%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
finish : 2016-11-21T17:20:50+0900
Minibatch loss at step 100: 8.607947
Minibatch accuracy_length: 46.9%


In [36]:
# Minibatch loss at step 10000: 0.981744
# Minibatch accuracy_length: 100.0%
# Minibatch accuracy_digit_0: 95.3%
# Minibatch accuracy_digit_1: 95.3%
# Minibatch accuracy_digit_2: 91.7%
# Minibatch accuracy_digit_3: 97.8%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 84.4%
# finish : 2016-11-21T17:48:14+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=10001, 
                        batch_size=64,  
                        num_convs=[16,32,64,0,0,0,0,0], 
                        num_fc_1=64, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(64, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(64, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(64, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 14.046442
Minibatch accuracy_length: 1.6%
Minibatch accuracy_digit_0: 18.8%
Minibatch accuracy_digit_1: 4.7%
Minibatch accuracy_digit_2: 3.3%
Minibatch accuracy_digit_3: 7.1%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
finish : 2016-11-21T17:37:57+0900
Minibatch loss at step 500: 7.043739
Minibatch accuracy_length: 59.4%
Minibatch accuracy_digit_0: 29.7%
Minibatch accuracy_digit_1: 32.8%
Minibatch accuracy_digit_2: 75.0%
Minibatch accuracy_digit_3: 85.7%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 3.1%
finish : 2016-11-21T17:38:23+0900
Minibatch loss at step 1000: 8.200775
Minibatch accuracy_length: 57.8

In [37]:
# Minibatch loss at step 10000: 1.042102
# Minibatch accuracy_length: 98.4%
# Minibatch accuracy_digit_0: 96.9%
# Minibatch accuracy_digit_1: 93.8%
# Minibatch accuracy_digit_2: 93.3%
# Minibatch accuracy_digit_3: 95.6%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 85.9%
# finish : 2016-11-21T17:59:46+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=10001, 
                        batch_size=64,  
                        num_convs=[16,32,64,64,0,0,0,0], 
                        num_fc_1=128, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(64, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(64, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(64, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 15.082737
Minibatch accuracy_length: 40.6%
Minibatch accuracy_digit_0: 7.8%
Minibatch accuracy_digit_1: 6.2%
Minibatch accuracy_digit_2: 0.0%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
finish : 2016-11-21T17:48:42+0900
Minibatch loss at step 500: 7.625749
Minibatch accuracy_length: 59.4%
Minibatch accuracy_digit_0: 29.7%
Minibatch accuracy_digit_1: 32.8%
Minibatch accuracy_digit_2: 75.0%
Minibatch accuracy_digit_3: 85.7%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 3.1%
finish : 2016-11-21T17:49:12+0900
Minibatch loss at step 1000: 8.617565
Minibatch accuracy_length: 59.4

In [38]:
# Minibatch loss at step 10000: 1.228529
# Minibatch accuracy_length: 100.0%
# Minibatch accuracy_digit_0: 93.8%
# Minibatch accuracy_digit_1: 92.2%
# Minibatch accuracy_digit_2: 90.0%
# Minibatch accuracy_digit_3: 97.8%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 79.7%
# finish : 2016-11-21T18:16:25+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=10001, 
                        batch_size=64,  
                        num_convs=[16,32,64,128,0,0,0,0], 
                        num_fc_1=128, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(64, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(64, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(64, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 15.264286
Minibatch accuracy_length: 6.2%
Minibatch accuracy_digit_0: 4.7%
Minibatch accuracy_digit_1: 3.1%
Minibatch accuracy_digit_2: 76.7%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
finish : 2016-11-21T17:59:51+0900
Minibatch loss at step 500: 8.695004
Minibatch accuracy_length: 59.4%
Minibatch accuracy_digit_0: 29.7%
Minibatch accuracy_digit_1: 32.8%
Minibatch accuracy_digit_2: 75.0%
Minibatch accuracy_digit_3: 85.7%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 3.1%
finish : 2016-11-21T18:00:40+0900
Minibatch loss at step 1000: 7.883448
Minibatch accuracy_length: 90.6

In [39]:
# Minibatch loss at step 10000: 1.438988
# Minibatch accuracy_length: 96.9%
# Minibatch accuracy_digit_0: 92.2%
# Minibatch accuracy_digit_1: 92.2%
# Minibatch accuracy_digit_2: 90.0%
# Minibatch accuracy_digit_3: 97.8%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 81.2%
# finish : 2016-11-21T18:40:10+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=10001, 
                        batch_size=64,  
                        num_convs=[16,32,64,128,192,0,0,0], 
                        num_fc_1=128, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(64, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(64, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(64, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 21.050270
Minibatch accuracy_length: 6.2%
Minibatch accuracy_digit_0: 4.7%
Minibatch accuracy_digit_1: 6.2%
Minibatch accuracy_digit_2: 1.7%
Minibatch accuracy_digit_3: 7.1%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
finish : 2016-11-21T18:16:31+0900
Minibatch loss at step 500: 9.943953
Minibatch accuracy_length: 59.4%
Minibatch accuracy_digit_0: 29.7%
Minibatch accuracy_digit_1: 32.8%
Minibatch accuracy_digit_2: 75.0%
Minibatch accuracy_digit_3: 85.7%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 3.1%
finish : 2016-11-21T18:17:39+0900
Minibatch loss at step 1000: 10.823154
Minibatch accuracy_length: 57.8

In [40]:
# Minibatch loss at step 10000: 1.814906
# Minibatch accuracy_length: 96.9%
# Minibatch accuracy_digit_0: 90.6%
# Minibatch accuracy_digit_1: 92.2%
# Minibatch accuracy_digit_2: 81.7%
# Minibatch accuracy_digit_3: 97.8%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 76.6%
# finish : 2016-11-21T19:04:15+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=10001, 
                        batch_size=64,  
                        num_convs=[16,32,64,128,192,192,0,0], 
                        num_fc_1=128, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(64, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(64, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(64, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 16.536842
Minibatch accuracy_length: 29.7%
Minibatch accuracy_digit_0: 9.4%
Minibatch accuracy_digit_1: 17.2%
Minibatch accuracy_digit_2: 55.0%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
finish : 2016-11-21T18:40:16+0900
Minibatch loss at step 500: 9.942520
Minibatch accuracy_length: 59.4%
Minibatch accuracy_digit_0: 29.7%
Minibatch accuracy_digit_1: 32.8%
Minibatch accuracy_digit_2: 75.0%
Minibatch accuracy_digit_3: 85.7%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 3.1%
finish : 2016-11-21T18:41:27+0900
Minibatch loss at step 1000: 10.829155
Minibatch accuracy_length: 5

In [41]:
# Minibatch loss at step 10000: 2.061375
# Minibatch accuracy_length: 95.3%
# Minibatch accuracy_digit_0: 89.1%
# Minibatch accuracy_digit_1: 82.8%
# Minibatch accuracy_digit_2: 85.0%
# Minibatch accuracy_digit_3: 88.9%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 68.8%
# finish : 2016-11-21T19:29:01+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=10001, 
                        batch_size=64,  
                        num_convs=[16,32,64,128,192,192,192,0], 
                        num_fc_1=128, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(64, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(64, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(64, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 21.439852
Minibatch accuracy_length: 0.0%
Minibatch accuracy_digit_0: 4.7%
Minibatch accuracy_digit_1: 3.1%
Minibatch accuracy_digit_2: 3.3%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
finish : 2016-11-21T19:04:21+0900
Minibatch loss at step 500: 9.939278
Minibatch accuracy_length: 59.4%
Minibatch accuracy_digit_0: 29.7%
Minibatch accuracy_digit_1: 32.8%
Minibatch accuracy_digit_2: 75.0%
Minibatch accuracy_digit_3: 85.7%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 3.1%
finish : 2016-11-21T19:05:35+0900
Minibatch loss at step 1000: 10.818617
Minibatch accuracy_length: 57.8

In [42]:
# Minibatch loss at step 10000: 1.997093
# Minibatch accuracy_length: 96.9%
# Minibatch accuracy_digit_0: 87.5%
# Minibatch accuracy_digit_1: 75.0%
# Minibatch accuracy_digit_2: 80.0%
# Minibatch accuracy_digit_3: 88.9%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 60.9%
# finish : 2016-11-22T03:32:31+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=10001, 
                        batch_size=64,  
                        num_convs=[16,32,32,32,32,32,32,32], 
                        num_fc_1=64, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(64, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(64, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(64, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 13.853424
Minibatch accuracy_length: 0.0%
Minibatch accuracy_digit_0: 0.0%
Minibatch accuracy_digit_1: 7.8%
Minibatch accuracy_digit_2: 3.3%
Minibatch accuracy_digit_3: 64.3%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
finish : 2016-11-22T03:22:28+0900
Minibatch loss at step 500: 6.764676
Minibatch accuracy_length: 59.4%
Minibatch accuracy_digit_0: 29.7%
Minibatch accuracy_digit_1: 32.8%
Minibatch accuracy_digit_2: 75.0%
Minibatch accuracy_digit_3: 85.7%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 3.1%
finish : 2016-11-22T03:22:59+0900
Minibatch loss at step 1000: 8.013912
Minibatch accuracy_length: 57.8

In [45]:
# Minibatch loss at step 10000: 2.566642
# Minibatch accuracy_length: 93.8%
# Minibatch accuracy_digit_0: 82.8%
# Minibatch accuracy_digit_1: 76.6%
# Minibatch accuracy_digit_2: 71.7%
# Minibatch accuracy_digit_3: 93.3%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 59.4%
# finish : 2016-11-22T03:51:04+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32, use_standard_score=True)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=10001, 
                        batch_size=64,  
                        num_convs=[16,32,64,0,0,0,0,0], 
                        num_fc_1=64, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_True.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(64, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(64, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(64, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 14.288710
Minibatch accuracy_length: 3.1%
Minibatch accuracy_digit_0: 15.6%
Minibatch accuracy_digit_1: 7.8%
Minibatch accuracy_digit_2: 0.0%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
finish : 2016-11-22T03:39:49+0900
Minibatch loss at step 500: 6.509155
Minibatch accuracy_length: 67.2%
Minibatch accuracy_digit_0: 23.4%
Minibatch accuracy_digit_1: 32.8%
Minibatch accuracy_digit_2: 75.0%
Minibatch accuracy_digit_3: 85.7%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 3.1%
finish : 2016-11-22T03:40:24+0900
Minibatch loss at step 1000: 7.057590
Minibatch accuracy_length: 76.6%

In [46]:
# Minibatch loss at step 3000: 3.319476
# Minibatch accuracy_length: 89.1%
# Minibatch accuracy_digit_0: 78.1%
# Minibatch accuracy_digit_1: 62.5%
# Minibatch accuracy_digit_2: 77.0%
# Minibatch accuracy_digit_3: 94.1%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 40.6%
# finish : 2016-11-22T03:55:35+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32, use_standard_score=True)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=3001, 
                        batch_size=64,  
                        num_convs=[16,32,64,64,0,0,0,0], 
                        num_fc_1=64, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_True.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(64, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(64, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(64, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 13.803230
Minibatch accuracy_length: 43.8%
Minibatch accuracy_digit_0: 7.8%
Minibatch accuracy_digit_1: 7.8%
Minibatch accuracy_digit_2: 3.3%
Minibatch accuracy_digit_3: 14.3%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
finish : 2016-11-22T03:52:13+0900
Minibatch loss at step 500: 6.809325
Minibatch accuracy_length: 59.4%
Minibatch accuracy_digit_0: 29.7%
Minibatch accuracy_digit_1: 32.8%
Minibatch accuracy_digit_2: 75.0%
Minibatch accuracy_digit_3: 85.7%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 3.1%
finish : 2016-11-22T03:52:50+0900
Minibatch loss at step 1000: 7.455029
Minibatch accuracy_length: 64

In [47]:
# Minibatch loss at step 3000: 4.487931
# Minibatch accuracy_length: 90.6%
# Minibatch accuracy_digit_0: 53.1%
# Minibatch accuracy_digit_1: 53.1%
# Minibatch accuracy_digit_2: 66.7%
# Minibatch accuracy_digit_3: 100.0%
# Minibatch accuracy_digit_4: 0.0%
# Minibatch accuracy: 15.6%
# finish : 2016-11-22T03:57:24+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32, use_standard_score=True)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=3001, 
                        batch_size=32,  
                        num_convs=[16,32,64,64,0,0,0,0], 
                        num_fc_1=64, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_True.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(32, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(32, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(32, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 13.837033
Minibatch accuracy_length: 43.8%
Minibatch accuracy_digit_0: 0.0%
Minibatch accuracy_digit_1: 6.2%
Minibatch accuracy_digit_2: 0.0%
Minibatch accuracy_digit_3: 14.3%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
finish : 2016-11-22T03:55:41+0900
Minibatch loss at step 500: 7.993669
Minibatch accuracy_length: 37.5%
Minibatch accuracy_digit_0: 18.8%
Minibatch accuracy_digit_1: 34.4%
Minibatch accuracy_digit_2: 63.6%
Minibatch accuracy_digit_3: 60.0%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
finish : 2016-11-22T03:55:58+0900
Minibatch loss at step 1000: 6.480633
Minibatch accuracy_length: 65.6

In [48]:
# Minibatch loss at step 3000: 6.703593
# Minibatch accuracy_length: 81.2%
# Minibatch accuracy_digit_0: 43.8%
# Minibatch accuracy_digit_1: 25.0%
# Minibatch accuracy_digit_2: 56.2%
# Minibatch accuracy_digit_3: 88.9%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 0.0%
# finish : 2016-11-22T03:58:53+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32, use_standard_score=True)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=3001, 
                        batch_size=16,  
                        num_convs=[16,32,64,64,0,0,0,0], 
                        num_fc_1=64, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_True.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(16, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(16, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(16, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 13.817432
Minibatch accuracy_length: 6.2%
Minibatch accuracy_digit_0: 6.2%
Minibatch accuracy_digit_1: 6.2%
Minibatch accuracy_digit_2: 0.0%
Minibatch accuracy_digit_3: 50.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
finish : 2016-11-22T03:57:29+0900
Minibatch loss at step 500: 7.142085
Minibatch accuracy_length: 50.0%
Minibatch accuracy_digit_0: 37.5%
Minibatch accuracy_digit_1: 31.2%
Minibatch accuracy_digit_2: 61.5%
Minibatch accuracy_digit_3: 100.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
finish : 2016-11-22T03:57:43+0900
Minibatch loss at step 1000: 8.052161
Minibatch accuracy_length: 37.5%


In [49]:
# Minibatch loss at step 3000: 7.446798
# Minibatch accuracy_length: 62.5%
# Minibatch accuracy_digit_0: 50.0%
# Minibatch accuracy_digit_1: 12.5%
# Minibatch accuracy_digit_2: 25.0%
# Minibatch accuracy_digit_3: 100.0%
# Minibatch accuracy_digit_4: 0.0%
# Minibatch accuracy: 0.0%
# finish : 2016-11-22T04:00:25+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32, use_standard_score=True)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=3001, 
                        batch_size=8,  
                        num_convs=[16,32,64,64,0,0,0,0], 
                        num_fc_1=64, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_True.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(8, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(8, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(8, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 14.154558
Minibatch accuracy_length: 0.0%
Minibatch accuracy_digit_0: 12.5%
Minibatch accuracy_digit_1: 12.5%
Minibatch accuracy_digit_2: 0.0%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
finish : 2016-11-22T03:58:58+0900
Minibatch loss at step 500: 7.323334
Minibatch accuracy_length: 37.5%
Minibatch accuracy_digit_0: 50.0%
Minibatch accuracy_digit_1: 25.0%
Minibatch accuracy_digit_2: 50.0%
Minibatch accuracy_digit_3: 100.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
finish : 2016-11-22T03:59:12+0900
Minibatch loss at step 1000: 6.759316
Minibatch accuracy_length: 50.0%
Mi

In [50]:
# Minibatch loss at step 3000: 1.589227
# Minibatch accuracy_length: 95.3%
# Minibatch accuracy_digit_0: 93.8%
# Minibatch accuracy_digit_1: 89.1%
# Minibatch accuracy_digit_2: 90.2%
# Minibatch accuracy_digit_3: 94.1%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 76.6%
# finish : 2016-11-22T04:04:40+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=3001, 
                        batch_size=64,  
                        num_convs=[16,32,64,0,0,0,0,0], 
                        num_fc_1=64, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(64, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(64, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(64, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 14.017056
Minibatch accuracy_length: 1.6%
Minibatch accuracy_digit_0: 10.9%
Minibatch accuracy_digit_1: 9.4%
Minibatch accuracy_digit_2: 3.3%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
finish : 2016-11-22T04:02:05+0900
Minibatch loss at step 500: 6.705099
Minibatch accuracy_length: 59.4%
Minibatch accuracy_digit_0: 29.7%
Minibatch accuracy_digit_1: 32.8%
Minibatch accuracy_digit_2: 75.0%
Minibatch accuracy_digit_3: 85.7%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 3.1%
finish : 2016-11-22T04:02:28+0900
Minibatch loss at step 1000: 5.662883
Minibatch accuracy_length: 90

In [51]:
# Minibatch loss at step 3000: 1.984119
# Minibatch accuracy_length: 96.9%
# Minibatch accuracy_digit_0: 87.5%
# Minibatch accuracy_digit_1: 79.7%
# Minibatch accuracy_digit_2: 88.5%
# Minibatch accuracy_digit_3: 94.1%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 59.4%
# finish : 2016-11-22T04:08:16+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=3001, 
                        batch_size=64,  
                        num_convs=[16,32,64,0,0,0,0,0], 
                        num_fc_1=64, 
                        num_fc_2=32,
                       ).run()

svhn/reader/svhn_5_32_300000_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(64, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(64, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(64, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 14.588334
Minibatch accuracy_length: 71.9%
Minibatch accuracy_digit_0: 4.7%
Minibatch accuracy_digit_1: 12.5%
Minibatch accuracy_digit_2: 0.0%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
finish : 2016-11-22T04:05:38+0900
Minibatch loss at step 500: 7.027736
Minibatch accuracy_length: 59.4%
Minibatch accuracy_digit_0: 29.7%
Minibatch accuracy_digit_1: 32.8%
Minibatch accuracy_digit_2: 75.0%
Minibatch accuracy_digit_3: 85.7%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 3.1%
finish : 2016-11-22T04:06:02+0900
Minibatch loss at step 1000: 7.914276
Minibatch accuracy_length: 62.

In [52]:
# Minibatch loss at step 3000: 2.182108
# Minibatch accuracy_length: 96.9%
# Minibatch accuracy_digit_0: 89.1%
# Minibatch accuracy_digit_1: 84.4%
# Minibatch accuracy_digit_2: 86.9%
# Minibatch accuracy_digit_3: 94.1%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 60.9%
# finish : 2016-11-22T04:12:28+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=3001, 
                        batch_size=64,  
                        num_convs=[16,32,64,0,0,0,0,0], 
                        num_fc_1=128, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(64, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(64, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(64, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 14.817883
Minibatch accuracy_length: 0.0%
Minibatch accuracy_digit_0: 1.6%
Minibatch accuracy_digit_1: 15.6%
Minibatch accuracy_digit_2: 3.3%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
finish : 2016-11-22T04:09:14+0900
Minibatch loss at step 500: 7.635503
Minibatch accuracy_length: 59.4%
Minibatch accuracy_digit_0: 29.7%
Minibatch accuracy_digit_1: 32.8%
Minibatch accuracy_digit_2: 75.0%
Minibatch accuracy_digit_3: 85.7%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 3.1%
finish : 2016-11-22T04:09:45+0900
Minibatch loss at step 1000: 8.781966
Minibatch accuracy_length: 57.8

In [53]:
# Minibatch loss at step 3000: 1.842490
# Minibatch accuracy_length: 93.8%
# Minibatch accuracy_digit_0: 92.2%
# Minibatch accuracy_digit_1: 89.1%
# Minibatch accuracy_digit_2: 93.4%
# Minibatch accuracy_digit_3: 94.1%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 76.6%
# finish : 2016-11-22T04:18:43+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=3001, 
                        batch_size=64,  
                        num_convs=[16,32,64,64,0,0,0,0], 
                        num_fc_1=128, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(64, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(64, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(64, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 14.589385
Minibatch accuracy_length: 71.9%
Minibatch accuracy_digit_0: 3.1%
Minibatch accuracy_digit_1: 9.4%
Minibatch accuracy_digit_2: 3.3%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 1.6%
finish : 2016-11-22T04:15:30+0900
Minibatch loss at step 500: 7.616387
Minibatch accuracy_length: 59.4%
Minibatch accuracy_digit_0: 29.7%
Minibatch accuracy_digit_1: 32.8%
Minibatch accuracy_digit_2: 75.0%
Minibatch accuracy_digit_3: 85.7%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 3.1%
finish : 2016-11-22T04:16:01+0900
Minibatch loss at step 1000: 8.605742
Minibatch accuracy_length: 59

In [54]:
# Minibatch loss at step 10000: 1.553630
# Minibatch accuracy_length: 100.0%
# Minibatch accuracy_digit_0: 93.8%
# Minibatch accuracy_digit_1: 93.8%
# Minibatch accuracy_digit_2: 91.7%
# Minibatch accuracy_digit_3: 95.6%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 78.1%
# finish : 2016-11-22T04:48:12+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=10001, 
                        batch_size=64,  
                        num_convs=[16,32,64,128,0,0,0,0], 
                        num_fc_1=256, 
                        num_fc_2=128,
                       ).run()

svhn/reader/svhn_5_32_300000_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(64, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(64, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(64, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 18.992428
Minibatch accuracy_length: 6.2%
Minibatch accuracy_digit_0: 9.4%
Minibatch accuracy_digit_1: 15.6%
Minibatch accuracy_digit_2: 76.7%
Minibatch accuracy_digit_3: 7.1%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
finish : 2016-11-22T04:20:18+0900
Minibatch loss at step 500: 11.054802
Minibatch accuracy_length: 59.4%
Minibatch accuracy_digit_0: 29.7%
Minibatch accuracy_digit_1: 32.8%
Minibatch accuracy_digit_2: 75.0%
Minibatch accuracy_digit_3: 85.7%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 3.1%
finish : 2016-11-22T04:21:39+0900
Minibatch loss at step 1000: 11.089961
Minibatch accuracy_length: 7

In [55]:
# Minibatch loss at step 10000: 1.021927
# Minibatch accuracy_length: 100.0%
# Minibatch accuracy_digit_0: 95.3%
# Minibatch accuracy_digit_1: 93.8%
# Minibatch accuracy_digit_2: 91.7%
# Minibatch accuracy_digit_3: 97.8%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 82.8%
# finish : 2016-11-22T04:58:48+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=10001, 
                        batch_size=64,  
                        num_convs=[16,32,64,64,0,0,0,0], 
                        num_fc_1=128, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(64, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(64, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(64, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 14.921124
Minibatch accuracy_length: 46.9%
Minibatch accuracy_digit_0: 14.1%
Minibatch accuracy_digit_1: 18.8%
Minibatch accuracy_digit_2: 0.0%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 3.1%
finish : 2016-11-22T04:48:17+0900
Minibatch loss at step 500: 7.635847
Minibatch accuracy_length: 59.4%
Minibatch accuracy_digit_0: 29.7%
Minibatch accuracy_digit_1: 32.8%
Minibatch accuracy_digit_2: 75.0%
Minibatch accuracy_digit_3: 85.7%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 3.1%
finish : 2016-11-22T04:48:47+0900
Minibatch loss at step 1000: 8.781432
Minibatch accuracy_length: 

In [56]:
# Minibatch loss at step 10000: 0.985925
# Minibatch accuracy_length: 97.7%
# Minibatch accuracy_digit_0: 99.2%
# Minibatch accuracy_digit_1: 93.0%
# Minibatch accuracy_digit_2: 93.7%
# Minibatch accuracy_digit_3: 98.7%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 88.3%
# finish : 2016-11-22T05:14:55+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=10001, 
                        batch_size=128,  
                        num_convs=[16,32,64,64,0,0,0,0], 
                        num_fc_1=128, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(128, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(128, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(128, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 14.721541
Minibatch accuracy_length: 10.9%
Minibatch accuracy_digit_0: 1.6%
Minibatch accuracy_digit_1: 11.7%
Minibatch accuracy_digit_2: 4.4%
Minibatch accuracy_digit_3: 67.7%
Minibatch accuracy_digit_4: 16.7%
Minibatch accuracy: 0.0%
finish : 2016-11-22T04:58:54+0900
Minibatch loss at step 500: 8.501460
Minibatch accuracy_length: 54.7%
Minibatch accuracy_digit_0: 28.9%
Minibatch accuracy_digit_1: 16.4%
Minibatch accuracy_digit_2: 44.0%
Minibatch accuracy_digit_3: 91.6%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.8%
finish : 2016-11-22T04:59:36+0900
Minibatch loss at step 1000: 5.788251
Minibatch accuracy_length

In [57]:
# Minibatch loss at step 10000: 1.005622
# Minibatch accuracy_length: 99.2%
# Minibatch accuracy_digit_0: 93.0%
# Minibatch accuracy_digit_1: 93.8%
# Minibatch accuracy_digit_2: 93.7%
# Minibatch accuracy_digit_3: 94.9%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 80.5%
# finish : 2016-11-22T05:32:12+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=10001, 
                        batch_size=128,  
                        num_convs=[16,16,32,64,64,0,0,0], 
                        num_fc_1=128, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(128, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(128, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(128, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 14.977258
Minibatch accuracy_length: 3.9%
Minibatch accuracy_digit_0: 13.3%
Minibatch accuracy_digit_1: 7.0%
Minibatch accuracy_digit_2: 4.4%
Minibatch accuracy_digit_3: 3.2%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.8%
finish : 2016-11-22T05:15:01+0900
Minibatch loss at step 500: 8.615595
Minibatch accuracy_length: 57.0%
Minibatch accuracy_digit_0: 28.9%
Minibatch accuracy_digit_1: 16.4%
Minibatch accuracy_digit_2: 44.0%
Minibatch accuracy_digit_3: 91.6%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
finish : 2016-11-22T05:15:50+0900
Minibatch loss at step 1000: 8.109586
Minibatch accuracy_length: 4

In [58]:
# Minibatch loss at step 10000: 1.413344
# Minibatch accuracy_length: 96.9%
# Minibatch accuracy_digit_0: 86.7%
# Minibatch accuracy_digit_1: 89.8%
# Minibatch accuracy_digit_2: 87.3%
# Minibatch accuracy_digit_3: 93.7%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 72.7%
# finish : 2016-11-22T05:48:05+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=10001, 
                        batch_size=128,  
                        num_convs=[16,16,16,32,64,64,0,0], 
                        num_fc_1=128, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(128, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(128, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(128, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 14.619118
Minibatch accuracy_length: 19.5%
Minibatch accuracy_digit_0: 14.8%
Minibatch accuracy_digit_1: 13.3%
Minibatch accuracy_digit_2: 4.4%
Minibatch accuracy_digit_3: 3.2%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
finish : 2016-11-22T05:32:18+0900
Minibatch loss at step 500: 8.614301
Minibatch accuracy_length: 57.0%
Minibatch accuracy_digit_0: 28.9%
Minibatch accuracy_digit_1: 16.4%
Minibatch accuracy_digit_2: 44.0%
Minibatch accuracy_digit_3: 91.6%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
finish : 2016-11-22T05:33:03+0900
Minibatch loss at step 1000: 8.113678
Minibatch accuracy_length:

In [59]:
# Minibatch loss at step 10000: 1.493153
# Minibatch accuracy_length: 95.3%
# Minibatch accuracy_digit_0: 88.3%
# Minibatch accuracy_digit_1: 89.8%
# Minibatch accuracy_digit_2: 88.9%
# Minibatch accuracy_digit_3: 92.4%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 71.9%
# finish : 2016-11-22T06:06:18+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=10001, 
                        batch_size=128,  
                        num_convs=[16,16,16,16,32,64,64,0], 
                        num_fc_1=128, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(128, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(128, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(128, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 14.618623
Minibatch accuracy_length: 4.7%
Minibatch accuracy_digit_0: 31.2%
Minibatch accuracy_digit_1: 7.8%
Minibatch accuracy_digit_2: 17.5%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
finish : 2016-11-22T05:48:11+0900
Minibatch loss at step 500: 8.614834
Minibatch accuracy_length: 57.0%
Minibatch accuracy_digit_0: 28.9%
Minibatch accuracy_digit_1: 16.4%
Minibatch accuracy_digit_2: 44.0%
Minibatch accuracy_digit_3: 91.6%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
finish : 2016-11-22T05:48:59+0900
Minibatch loss at step 1000: 8.108915
Minibatch accuracy_length: 

In [60]:
# Minibatch loss at step 10000: 1.360078
# Minibatch accuracy_length: 96.1%
# Minibatch accuracy_digit_0: 93.8%
# Minibatch accuracy_digit_1: 90.6%
# Minibatch accuracy_digit_2: 87.3%
# Minibatch accuracy_digit_3: 93.7%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 79.7%
# finish : 2016-11-22T06:24:03+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=10001, 
                        batch_size=128,  
                        num_convs=[16,16,16,16,16,32,64,64], 
                        num_fc_1=128, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(128, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(128, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(128, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 14.567421
Minibatch accuracy_length: 64.8%
Minibatch accuracy_digit_0: 19.5%
Minibatch accuracy_digit_1: 5.5%
Minibatch accuracy_digit_2: 24.6%
Minibatch accuracy_digit_3: 3.2%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.8%
finish : 2016-11-22T06:06:24+0900
Minibatch loss at step 500: 8.615663
Minibatch accuracy_length: 57.0%
Minibatch accuracy_digit_0: 28.9%
Minibatch accuracy_digit_1: 16.4%
Minibatch accuracy_digit_2: 44.0%
Minibatch accuracy_digit_3: 91.6%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
finish : 2016-11-22T06:07:18+0900
Minibatch loss at step 1000: 8.120823
Minibatch accuracy_length:

In [63]:
# Minibatch loss at step 10000: 1.674805
# Minibatch accuracy_length: 96.9%
# Minibatch accuracy_digit_0: 93.8%
# Minibatch accuracy_digit_1: 85.2%
# Minibatch accuracy_digit_2: 88.9%
# Minibatch accuracy_digit_3: 89.9%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 75.0%
# finish : 2016-11-22T23:05:21+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32, 
                                            use_standard_score=False, add_margin=True, margin_ratio=0.3, apply_gaussian_filter=True)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=10001, 
                        batch_size=128,  
                        num_convs=[16,32,64,64,0,0,0,0], 
                        num_fc_1=128, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False_True_0.3_True.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(128, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(128, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(128, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 15.266574
Minibatch accuracy_length: 64.8%
Minibatch accuracy_digit_0: 12.5%
Minibatch accuracy_digit_1: 10.2%
Minibatch accuracy_digit_2: 3.5%
Minibatch accuracy_digit_3: 3.2%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
finish : 2016-11-22T22:47:50+0900
Minibatch loss at step 500: 8.615908
Minibatch accuracy_length: 57.0%
Minibatch accuracy_digit_0: 28.9%
Minibatch accuracy_digit_1: 16.4%
Minibatch accuracy_digit_2: 44.0%
Minibatch accuracy_digit_3: 91.6%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
finish : 2016-11-22T22:48:40+0900
Minibatch loss at step 1000: 8.107287
Minibatch ac

In [64]:
# Minibatch loss at step 10000: 1.641402
# Minibatch accuracy_length: 94.5%
# Minibatch accuracy_digit_0: 90.6%
# Minibatch accuracy_digit_1: 90.6%
# Minibatch accuracy_digit_2: 92.9%
# Minibatch accuracy_digit_3: 92.4%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 79.7%
# finish : 2016-11-22T23:22:14+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32, 
                                            use_standard_score=False, add_margin=True, margin_ratio=0.3)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=10001, 
                        batch_size=128,  
                        num_convs=[16,32,64,64,0,0,0,0], 
                        num_fc_1=128, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False_True_0.3_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(128, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(128, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(128, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 16.599298
Minibatch accuracy_length: 2.3%
Minibatch accuracy_digit_0: 3.9%
Minibatch accuracy_digit_1: 5.5%
Minibatch accuracy_digit_2: 1.8%
Minibatch accuracy_digit_3: 3.2%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
finish : 2016-11-22T23:05:26+0900
Minibatch loss at step 500: 8.614979
Minibatch accuracy_length: 57.0%
Minibatch accuracy_digit_0: 28.9%
Minibatch accuracy_digit_1: 16.4%
Minibatch accuracy_digit_2: 44.0%
Minibatch accuracy_digit_3: 91.6%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
finish : 2016-11-22T23:06:10+0900
Minibatch loss at step 1000: 8.122225
Minibatch accu

In [65]:
# Minibatch loss at step 10000: 2.771159
# Minibatch accuracy_length: 93.8%
# Minibatch accuracy_digit_0: 81.2%
# Minibatch accuracy_digit_1: 75.8%
# Minibatch accuracy_digit_2: 81.7%
# Minibatch accuracy_digit_3: 92.4%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 61.7%
# finish : 2016-11-22T23:41:08+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32, 
                                            use_standard_score=True, add_margin=True, margin_ratio=0.3)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=10001, 
                        batch_size=128,  
                        num_convs=[16,32,64,64,0,0,0,0], 
                        num_fc_1=128, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_True_True_0.3_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(128, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(128, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(128, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 15.492907
Minibatch accuracy_length: 0.0%
Minibatch accuracy_digit_0: 4.7%
Minibatch accuracy_digit_1: 12.5%
Minibatch accuracy_digit_2: 1.8%
Minibatch accuracy_digit_3: 3.2%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
finish : 2016-11-22T23:22:19+0900
Minibatch loss at step 500: 7.914188
Minibatch accuracy_length: 73.4%
Minibatch accuracy_digit_0: 28.1%
Minibatch accuracy_digit_1: 16.4%
Minibatch accuracy_digit_2: 44.0%
Minibatch accuracy_digit_3: 91.6%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 1.6%
finish : 2016-11-22T23:23:17+0900
Minibatch loss at step 1000: 6.458669
Minibatch accu

In [66]:
# Minibatch loss at step 10000: 2.906052
# Minibatch accuracy_length: 88.3%
# Minibatch accuracy_digit_0: 78.9%
# Minibatch accuracy_digit_1: 77.3%
# Minibatch accuracy_digit_2: 82.5%
# Minibatch accuracy_digit_3: 89.9%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 63.3%
# finish : 2016-11-22T23:59:01+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32, 
                                            use_standard_score=True, add_margin=True, margin_ratio=0.3, apply_gaussian_filter=True)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=10001, 
                        batch_size=128,  
                        num_convs=[16,32,64,64,0,0,0,0], 
                        num_fc_1=128, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_True_True_0.3_True.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(128, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(128, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(128, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 14.959444
Minibatch accuracy_length: 3.1%
Minibatch accuracy_digit_0: 4.7%
Minibatch accuracy_digit_1: 10.9%
Minibatch accuracy_digit_2: 3.5%
Minibatch accuracy_digit_3: 3.2%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
finish : 2016-11-22T23:41:13+0900
Minibatch loss at step 500: 8.018556
Minibatch accuracy_length: 73.4%
Minibatch accuracy_digit_0: 29.7%
Minibatch accuracy_digit_1: 14.8%
Minibatch accuracy_digit_2: 44.0%
Minibatch accuracy_digit_3: 91.6%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.8%
finish : 2016-11-22T23:42:06+0900
Minibatch loss at step 1000: 6.107223
Minibatch accur

In [68]:
# Minibatch loss at step 10000: 0.888348
# Minibatch accuracy_length: 98.4%
# Minibatch accuracy_digit_0: 95.3%
# Minibatch accuracy_digit_1: 94.5%
# Minibatch accuracy_digit_2: 94.4%
# Minibatch accuracy_digit_3: 97.5%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 85.2%
# finish : 2016-11-23T00:40:47+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=10001, 
                        batch_size=128,  
                        num_convs=[16,32,64,64,0,0,0,0], 
                        num_fc_1=128, 
                        num_fc_2=64,
                       ).run()

Pickling svhn/reader/svhn_5_32_300000_False_False_0.1_False_position3.pickle.
('rows[5][0]', (3, 350, 741), dtype('uint8'))
('total_data', (248822, 32, 32, 3))
('total_label', (248822, 6))
start : 2016-11-22T23:59:46+0900
end : 2016-11-23T00:23:41+0900
('train_data', (223939, 32, 32, 3))
('train_label', (223939, 6))
('validation_data', (12441, 32, 32, 3))
('validation_label', (12441, 6))
('test_data', (12442, 32, 32, 3))
('test_label', (12442, 6))
tf_train_dataset : Tensor("Placeholder:0", shape=(128, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(128, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(128, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 15.318023
Minibatch accuracy_length: 0.0%
Minibatch accuracy_digit_0: 0.0%
Minibatch accuracy_digit_1: 17.2%
Minibatch accuracy_digit_2: 1.8%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
finish : 2016-11-23T00:23:4

In [72]:
# Minibatch loss at step 10000: 0.910038
# Minibatch accuracy_length: 97.7%
# Minibatch accuracy_digit_0: 94.5%
# Minibatch accuracy_digit_1: 94.5%
# Minibatch accuracy_digit_2: 93.7%
# Minibatch accuracy_digit_3: 96.2%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 86.7%
# finish : 2016-11-25T05:26:47+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        use_drop_out=True,
                        drop_out_rate=0.9375,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=10001, 
                        batch_size=128,  
                        num_convs=[16,32,64,64,0,0,0,0], 
                        num_fc_1=128, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False_False_0.1_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(128, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(128, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(128, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 16.119133
Minibatch accuracy_length: 4.7%
Minibatch accuracy_digit_0: 4.7%
Minibatch accuracy_digit_1: 7.0%
Minibatch accuracy_digit_2: 2.6%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
finish : 2016-11-25T05:09:01+0900
Minibatch loss at step 500: 8.619555
Minibatch accuracy_length: 57.0%
Minibatch accuracy_digit_0: 28.9%
Minibatch accuracy_digit_1: 16.4%
Minibatch accuracy_digit_2: 44.0%
Minibatch accuracy_digit_3: 91.6%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
finish : 2016-11-25T05:09:43+0900
Minibatch loss at step 1000: 8.069834
Minibatch acc

In [85]:
# Minibatch loss at step 10000: 1.013274
# Minibatch accuracy_length: 97.7%
# Minibatch accuracy_digit_0: 94.5%
# Minibatch accuracy_digit_1: 96.1%
# Minibatch accuracy_digit_2: 94.4%
# Minibatch accuracy_digit_3: 94.9%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 85.9%
# finish : 2016-11-25T05:48:37+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        use_drop_out=True,
                        drop_out_rate=0.9375,
                        use_LCN=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=10001, 
                        batch_size=128,  
                        num_convs=[16,32,64,64,0,0,0,0], 
                        num_fc_1=128, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False_False_0.1_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(128, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(128, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(128, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 14.627675
Minibatch accuracy_length: 0.0%
Minibatch accuracy_digit_0: 14.1%
Minibatch accuracy_digit_1: 9.4%
Minibatch accuracy_digit_2: 3.5%
Minibatch accuracy_digit_3: 41.9%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
finish : 2016-11-25T05:29:56+0900
Minibatch loss at step 500: 7.289536
Minibatch accuracy_length: 76.6%
Minibatch accuracy_digit_0: 50.0%
Minibatch accuracy_digit_1: 22.7%
Minibatch accuracy_digit_2: 44.8%
Minibatch accuracy_digit_3: 91.6%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 3.9%
finish : 2016-11-25T05:30:53+0900
Minibatch loss at step 1000: 4.838192
Minibatch a

In [86]:
# Minibatch loss at step 10000: 1.046704
# Minibatch accuracy_length: 96.9%
# Minibatch accuracy_digit_0: 94.5%
# Minibatch accuracy_digit_1: 93.8%
# Minibatch accuracy_digit_2: 92.1%
# Minibatch accuracy_digit_3: 94.9%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 85.9%
# finish : 2016-11-25T06:05:40+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        use_drop_out=True,
                        drop_out_rate=0.9375,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=10001, 
                        batch_size=128,  
                        num_convs=[16,32,32,32,32,0,0,0], 
                        num_fc_1=128, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False_False_0.1_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(128, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(128, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(128, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 14.097105
Minibatch accuracy_length: 6.2%
Minibatch accuracy_digit_0: 14.1%
Minibatch accuracy_digit_1: 22.7%
Minibatch accuracy_digit_2: 4.4%
Minibatch accuracy_digit_3: 3.2%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
finish : 2016-11-25T05:48:43+0900
Minibatch loss at step 500: 8.027782
Minibatch accuracy_length: 57.0%
Minibatch accuracy_digit_0: 28.9%
Minibatch accuracy_digit_1: 16.4%
Minibatch accuracy_digit_2: 44.0%
Minibatch accuracy_digit_3: 91.6%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
finish : 2016-11-25T05:49:27+0900
Minibatch loss at step 1000: 7.627937
Minibatch a

In [87]:
# Minibatch loss at step 10000: 1.002527
# Minibatch accuracy_length: 96.9%
# Minibatch accuracy_digit_0: 93.0%
# Minibatch accuracy_digit_1: 93.0%
# Minibatch accuracy_digit_2: 93.7%
# Minibatch accuracy_digit_3: 96.2%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 84.4%
# finish : 2016-11-25T06:23:38+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        use_drop_out=True,
                        drop_out_rate=0.8,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=10001, 
                        batch_size=128,  
                        num_convs=[16,32,32,32,32,0,0,0], 
                        num_fc_1=128, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False_False_0.1_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(128, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(128, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(128, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 14.311393
Minibatch accuracy_length: 5.5%
Minibatch accuracy_digit_0: 10.2%
Minibatch accuracy_digit_1: 8.6%
Minibatch accuracy_digit_2: 2.6%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
finish : 2016-11-25T06:05:46+0900
Minibatch loss at step 500: 8.057206
Minibatch accuracy_length: 57.0%
Minibatch accuracy_digit_0: 28.9%
Minibatch accuracy_digit_1: 16.4%
Minibatch accuracy_digit_2: 44.0%
Minibatch accuracy_digit_3: 91.6%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
finish : 2016-11-25T06:06:38+0900
Minibatch loss at step 1000: 7.619127
Minibatch ac

In [88]:
# Minibatch loss at step 10000: 0.811576
# Minibatch accuracy_length: 97.7%
# Minibatch accuracy_digit_0: 96.9%
# Minibatch accuracy_digit_1: 93.0%
# Minibatch accuracy_digit_2: 94.4%
# Minibatch accuracy_digit_3: 97.5%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 87.5%
# finish : 2016-11-25T06:41:24+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        use_drop_out=True,
                        drop_out_rate=0.7,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=10001, 
                        batch_size=128,  
                        num_convs=[16,32,32,32,32,0,0,0], 
                        num_fc_1=128, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False_False_0.1_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(128, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(128, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(128, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 14.042958
Minibatch accuracy_length: 6.2%
Minibatch accuracy_digit_0: 4.7%
Minibatch accuracy_digit_1: 11.7%
Minibatch accuracy_digit_2: 3.5%
Minibatch accuracy_digit_3: 6.5%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
finish : 2016-11-25T06:23:44+0900
Minibatch loss at step 500: 8.052016
Minibatch accuracy_length: 57.0%
Minibatch accuracy_digit_0: 28.9%
Minibatch accuracy_digit_1: 16.4%
Minibatch accuracy_digit_2: 44.0%
Minibatch accuracy_digit_3: 91.6%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
finish : 2016-11-25T06:24:41+0900
Minibatch loss at step 1000: 7.597677
Minibatch ac

In [89]:
# Minibatch loss at step 10000: 1.113248
# Minibatch accuracy_length: 95.3%
# Minibatch accuracy_digit_0: 92.2%
# Minibatch accuracy_digit_1: 93.8%
# Minibatch accuracy_digit_2: 94.4%
# Minibatch accuracy_digit_3: 93.7%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 83.6%
# finish : 2016-11-25T06:59:33+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        use_drop_out=True,
                        drop_out_rate=0.6,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=10001, 
                        batch_size=128,  
                        num_convs=[16,32,32,32,32,0,0,0], 
                        num_fc_1=128, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False_False_0.1_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(128, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(128, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(128, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 14.235067
Minibatch accuracy_length: 6.2%
Minibatch accuracy_digit_0: 6.2%
Minibatch accuracy_digit_1: 6.2%
Minibatch accuracy_digit_2: 2.6%
Minibatch accuracy_digit_3: 9.7%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
finish : 2016-11-25T06:41:30+0900
Minibatch loss at step 500: 8.051390
Minibatch accuracy_length: 57.0%
Minibatch accuracy_digit_0: 28.9%
Minibatch accuracy_digit_1: 16.4%
Minibatch accuracy_digit_2: 44.0%
Minibatch accuracy_digit_3: 91.6%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
finish : 2016-11-25T06:42:25+0900
Minibatch loss at step 1000: 7.708331
Minibatch acc

In [90]:
# Minibatch loss at step 10000: 1.247436
# Minibatch accuracy_length: 96.1%
# Minibatch accuracy_digit_0: 95.3%
# Minibatch accuracy_digit_1: 89.8%
# Minibatch accuracy_digit_2: 88.1%
# Minibatch accuracy_digit_3: 92.4%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 78.9%
# finish : 2016-11-25T07:17:15+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        use_drop_out=True,
                        drop_out_rate=0.5,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=10001, 
                        batch_size=128,  
                        num_convs=[16,32,32,32,32,0,0,0], 
                        num_fc_1=128, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False_False_0.1_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(128, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(128, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(128, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 14.340179
Minibatch accuracy_length: 4.7%
Minibatch accuracy_digit_0: 7.8%
Minibatch accuracy_digit_1: 9.4%
Minibatch accuracy_digit_2: 3.5%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
finish : 2016-11-25T06:59:39+0900
Minibatch loss at step 500: 8.035017
Minibatch accuracy_length: 57.0%
Minibatch accuracy_digit_0: 28.9%
Minibatch accuracy_digit_1: 16.4%
Minibatch accuracy_digit_2: 44.0%
Minibatch accuracy_digit_3: 91.6%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
finish : 2016-11-25T07:00:26+0900
Minibatch loss at step 1000: 7.570957
Minibatch acc

In [91]:
# Minibatch loss at step 10000: 1.141586
# Minibatch accuracy_length: 96.9%
# Minibatch accuracy_digit_0: 89.8%
# Minibatch accuracy_digit_1: 92.2%
# Minibatch accuracy_digit_2: 92.1%
# Minibatch accuracy_digit_3: 96.2%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 78.1%
# finish : 2016-11-25T09:33:15+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        use_drop_out=True,
                        drop_out_rate=0.7,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=10001, 
                        batch_size=128,  
                        num_convs=[16,32,32,32,32,32,32,32], 
                        num_fc_1=128, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False_False_0.1_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(128, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(128, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(128, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 14.078024
Minibatch accuracy_length: 17.2%
Minibatch accuracy_digit_0: 14.1%
Minibatch accuracy_digit_1: 11.7%
Minibatch accuracy_digit_2: 2.6%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.8%
finish : 2016-11-25T09:17:17+0900
Minibatch loss at step 500: 8.053626
Minibatch accuracy_length: 57.0%
Minibatch accuracy_digit_0: 28.9%
Minibatch accuracy_digit_1: 16.4%
Minibatch accuracy_digit_2: 44.0%
Minibatch accuracy_digit_3: 91.6%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
finish : 2016-11-25T09:18:08+0900
Minibatch loss at step 1000: 7.605909
Minibatch 

In [92]:
# Minibatch loss at step 10000: 1.573483
# Minibatch accuracy_length: 98.4%
# Minibatch accuracy_digit_0: 95.3%
# Minibatch accuracy_digit_1: 96.9%
# Minibatch accuracy_digit_2: 92.1%
# Minibatch accuracy_digit_3: 94.9%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 86.7%
# finish : 2016-11-25T10:30:21+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        use_drop_out=True,
                        drop_out_rate=0.7,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=10001, 
                        batch_size=128,  
                        num_convs=[16,32,32,32,32,32,32,32], 
                        num_fc_1=1280, 
                        num_fc_2=640,
                       ).run()

svhn/reader/svhn_5_32_300000_False_False_0.1_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(128, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(128, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(128, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 21.936794
Minibatch accuracy_length: 46.1%
Minibatch accuracy_digit_0: 9.4%
Minibatch accuracy_digit_1: 6.2%
Minibatch accuracy_digit_2: 4.4%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
finish : 2016-11-25T09:56:12+0900
Minibatch loss at step 500: 14.970673
Minibatch accuracy_length: 57.0%
Minibatch accuracy_digit_0: 28.9%
Minibatch accuracy_digit_1: 16.4%
Minibatch accuracy_digit_2: 44.0%
Minibatch accuracy_digit_3: 91.6%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
finish : 2016-11-25T09:57:55+0900
Minibatch loss at step 1000: 13.936893
Minibatch 

In [94]:
# Minibatch loss at step 10000: 4.741112
# Minibatch accuracy_length: 95.3%
# Minibatch accuracy_digit_0: 87.5%
# Minibatch accuracy_digit_1: 77.3%
# Minibatch accuracy_digit_2: 81.7%
# Minibatch accuracy_digit_3: 88.6%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 59.4%
# finish : 2016-11-25T12:38:40+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        use_drop_out=True,
                        drop_out_rate=0.7,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=10001, 
                        batch_size=128,  
                        num_convs=[16,32,32,32,32,32,32,32], 
                        num_fc_1=6144, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False_False_0.1_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(128, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(128, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(128, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 45.964584
Minibatch accuracy_length: 18.8%
Minibatch accuracy_digit_0: 22.7%
Minibatch accuracy_digit_1: 9.4%
Minibatch accuracy_digit_2: 5.3%
Minibatch accuracy_digit_3: 3.2%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.8%
finish : 2016-11-25T10:32:49+0900
Minibatch loss at step 500: 35.844578
Minibatch accuracy_length: 57.0%
Minibatch accuracy_digit_0: 28.9%
Minibatch accuracy_digit_1: 16.4%
Minibatch accuracy_digit_2: 44.0%
Minibatch accuracy_digit_3: 91.6%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
finish : 2016-11-25T10:39:09+0900
Minibatch loss at step 1000: 32.162483
Minibatch

In [109]:
# Minibatch loss at step 10000: 0.858359
# Minibatch accuracy_length: 93.8%
# Minibatch accuracy_digit_0: 93.8%
# Minibatch accuracy_digit_1: 100.0%
# Minibatch accuracy_digit_2: 93.5%
# Minibatch accuracy_digit_3: 100.0%
# Minibatch accuracy_digit_4: 0.0%
# Minibatch accuracy: 84.4%
# finish : 2016-11-27T01:25:18+0900
            
# batch_size(=64) / 2
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=10001, 
                        batch_size=32,  
                        num_convs=[16,32,64,0,0,0,0,0], 
                        num_fc_1=64, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False_False_0.1_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(32, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(32, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(32, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 14.049033
Minibatch accuracy_length: 0.0%
Minibatch accuracy_digit_0: 9.4%
Minibatch accuracy_digit_1: 31.2%
Minibatch accuracy_digit_2: 6.5%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
finish : 2016-11-27T01:17:16+0900
Minibatch loss at step 500: 8.001852
Minibatch accuracy_length: 37.5%
Minibatch accuracy_digit_0: 18.8%
Minibatch accuracy_digit_1: 34.4%
Minibatch accuracy_digit_2: 63.6%
Minibatch accuracy_digit_3: 60.0%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
finish : 2016-11-27T01:17:38+0900
Minibatch loss at step 1000: 6.618875
Minibatch accur

In [110]:
# Minibatch loss at step 10000: 1.102812
# Minibatch accuracy_length: 98.4%
# Minibatch accuracy_digit_0: 93.0%
# Minibatch accuracy_digit_1: 92.2%
# Minibatch accuracy_digit_2: 90.5%
# Minibatch accuracy_digit_3: 94.9%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 83.6%
# finish : 2016-11-27T01:41:42+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=10001, 
                        batch_size=128,  
                        num_convs=[16,32,64,0,0,0,0,0], 
                        num_fc_1=64, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False_False_0.1_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(128, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(128, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(128, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 14.244916
Minibatch accuracy_length: 6.2%
Minibatch accuracy_digit_0: 1.6%
Minibatch accuracy_digit_1: 7.0%
Minibatch accuracy_digit_2: 1.8%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
finish : 2016-11-27T01:25:25+0900
Minibatch loss at step 500: 8.029725
Minibatch accuracy_length: 57.0%
Minibatch accuracy_digit_0: 28.9%
Minibatch accuracy_digit_1: 16.4%
Minibatch accuracy_digit_2: 44.0%
Minibatch accuracy_digit_3: 91.6%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
finish : 2016-11-27T01:26:09+0900
Minibatch loss at step 1000: 7.524869
Minibatch acc

In [111]:
# Minibatch loss at step 10000: 2.663368
# Minibatch accuracy_length: 87.5%
# Minibatch accuracy_digit_0: 100.0%
# Minibatch accuracy_digit_1: 75.0%
# Minibatch accuracy_digit_2: 78.6%
# Minibatch accuracy_digit_3: 100.0%
# Minibatch accuracy_digit_4: 0.0%
# Minibatch accuracy: 68.8%
# finish : 2016-11-27T01:49:52+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=10001, 
                        batch_size=16,  
                        num_convs=[16,32,64,0,0,0,0,0], 
                        num_fc_1=64, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False_False_0.1_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(16, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(16, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(16, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 14.421420
Minibatch accuracy_length: 0.0%
Minibatch accuracy_digit_0: 12.5%
Minibatch accuracy_digit_1: 6.2%
Minibatch accuracy_digit_2: 25.0%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
finish : 2016-11-27T01:41:48+0900
Minibatch loss at step 500: 7.111973
Minibatch accuracy_length: 50.0%
Minibatch accuracy_digit_0: 37.5%
Minibatch accuracy_digit_1: 31.2%
Minibatch accuracy_digit_2: 61.5%
Minibatch accuracy_digit_3: 100.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
finish : 2016-11-27T01:42:08+0900
Minibatch loss at step 1000: 7.822555
Minibatch accur

In [112]:
# Minibatch loss at step 10000: 5.794902
# Minibatch accuracy_length: 71.9%
# Minibatch accuracy_digit_0: 34.4%
# Minibatch accuracy_digit_1: 25.0%
# Minibatch accuracy_digit_2: 67.7%
# Minibatch accuracy_digit_3: 100.0%
# Minibatch accuracy_digit_4: 0.0%
# Minibatch accuracy: 0.0%
# finish : 2016-11-27T01:59:48+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.03,
                        digit_count=5,
                        num_channels=3,
                        num_steps=10001, 
                        batch_size=32,  
                        num_convs=[16,32,64,0,0,0,0,0], 
                        num_fc_1=64, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False_False_0.1_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(32, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(32, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(32, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 13.969602
Minibatch accuracy_length: 0.0%
Minibatch accuracy_digit_0: 18.8%
Minibatch accuracy_digit_1: 31.2%
Minibatch accuracy_digit_2: 3.2%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
finish : 2016-11-27T01:49:58+0900
Minibatch loss at step 500: 7.938001
Minibatch accuracy_length: 37.5%
Minibatch accuracy_digit_0: 18.8%
Minibatch accuracy_digit_1: 34.4%
Minibatch accuracy_digit_2: 63.6%
Minibatch accuracy_digit_3: 60.0%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
finish : 2016-11-27T01:50:29+0900
Minibatch loss at step 1000: 6.538775
Minibatch accu

In [113]:
# Minibatch loss at step 10000: 7.091837
# Minibatch accuracy_length: 43.8%
# Minibatch accuracy_digit_0: 18.8%
# Minibatch accuracy_digit_1: 6.2%
# Minibatch accuracy_digit_2: 58.1%
# Minibatch accuracy_digit_3: 100.0%
# Minibatch accuracy_digit_4: 0.0%
# Minibatch accuracy: 0.0%
# finish : 2016-11-27T02:07:38+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.09,
                        digit_count=5,
                        num_channels=3,
                        num_steps=10001, 
                        batch_size=32,  
                        num_convs=[16,32,64,0,0,0,0,0], 
                        num_fc_1=64, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False_False_0.1_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(32, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(32, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(32, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 14.221354
Minibatch accuracy_length: 3.1%
Minibatch accuracy_digit_0: 15.6%
Minibatch accuracy_digit_1: 3.1%
Minibatch accuracy_digit_2: 6.5%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
finish : 2016-11-27T01:59:55+0900
Minibatch loss at step 500: 8.053074
Minibatch accuracy_length: 37.5%
Minibatch accuracy_digit_0: 18.8%
Minibatch accuracy_digit_1: 34.4%
Minibatch accuracy_digit_2: 63.6%
Minibatch accuracy_digit_3: 60.0%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
finish : 2016-11-27T02:00:15+0900
Minibatch loss at step 1000: 6.525305
Minibatch accur

In [114]:
# Minibatch loss at step 10000: 9.348182
# Minibatch accuracy_length: 43.8%
# Minibatch accuracy_digit_0: 18.8%
# Minibatch accuracy_digit_1: 6.2%
# Minibatch accuracy_digit_2: 58.1%
# Minibatch accuracy_digit_3: 100.0%
# Minibatch accuracy_digit_4: 0.0%
# Minibatch accuracy: 0.0%
# finish : 2016-11-27T02:16:25+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.3,
                        digit_count=5,
                        num_channels=3,
                        num_steps=10001, 
                        batch_size=32,  
                        num_convs=[16,32,64,0,0,0,0,0], 
                        num_fc_1=64, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False_False_0.1_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(32, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(32, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(32, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 13.654697
Minibatch accuracy_length: 21.9%
Minibatch accuracy_digit_0: 25.0%
Minibatch accuracy_digit_1: 25.0%
Minibatch accuracy_digit_2: 58.1%
Minibatch accuracy_digit_3: 100.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
finish : 2016-11-27T02:07:44+0900
Minibatch loss at step 500: 13.329086
Minibatch accuracy_length: 37.5%
Minibatch accuracy_digit_0: 18.8%
Minibatch accuracy_digit_1: 34.4%
Minibatch accuracy_digit_2: 63.6%
Minibatch accuracy_digit_3: 60.0%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
finish : 2016-11-27T02:08:15+0900
Minibatch loss at step 1000: 11.114876
Minibatc

In [115]:
# Minibatch loss at step 10000: 0.991015
# Minibatch accuracy_length: 100.0%
# Minibatch accuracy_digit_0: 87.5%
# Minibatch accuracy_digit_1: 96.9%
# Minibatch accuracy_digit_2: 96.8%
# Minibatch accuracy_digit_3: 100.0%
# Minibatch accuracy_digit_4: 0.0%
# Minibatch accuracy: 81.2%
# finish : 2016-11-27T02:27:17+0900

# fc1 x 2
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=10001, 
                        batch_size=32,  
                        num_convs=[16,32,64,0,0,0,0,0], 
                        num_fc_1=128, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False_False_0.1_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(32, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(32, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(32, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 15.099500
Minibatch accuracy_length: 0.0%
Minibatch accuracy_digit_0: 6.2%
Minibatch accuracy_digit_1: 6.2%
Minibatch accuracy_digit_2: 0.0%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
finish : 2016-11-27T02:16:32+0900
Minibatch loss at step 500: 7.921667
Minibatch accuracy_length: 46.9%
Minibatch accuracy_digit_0: 15.6%
Minibatch accuracy_digit_1: 34.4%
Minibatch accuracy_digit_2: 63.6%
Minibatch accuracy_digit_3: 60.0%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
finish : 2016-11-27T02:17:03+0900
Minibatch loss at step 1000: 5.695154
Minibatch accura

In [116]:
# Minibatch loss at step 10000: 1.005498
# Minibatch accuracy_length: 96.9%
# Minibatch accuracy_digit_0: 93.8%
# Minibatch accuracy_digit_1: 96.9%
# Minibatch accuracy_digit_2: 96.8%
# Minibatch accuracy_digit_3: 100.0%
# Minibatch accuracy_digit_4: 0.0%
# Minibatch accuracy: 87.5%
# finish : 2016-11-27T02:44:05+0900

# fc1(=64) x 2^2
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=10001, 
                        batch_size=32,  
                        num_convs=[16,32,64,0,0,0,0,0], 
                        num_fc_1=256, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False_False_0.1_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(32, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(32, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(32, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 17.110712
Minibatch accuracy_length: 0.0%
Minibatch accuracy_digit_0: 3.1%
Minibatch accuracy_digit_1: 12.5%
Minibatch accuracy_digit_2: 6.5%
Minibatch accuracy_digit_3: 85.7%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
finish : 2016-11-27T02:27:22+0900
Minibatch loss at step 500: 9.863612
Minibatch accuracy_length: 37.5%
Minibatch accuracy_digit_0: 18.8%
Minibatch accuracy_digit_1: 34.4%
Minibatch accuracy_digit_2: 63.6%
Minibatch accuracy_digit_3: 60.0%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
finish : 2016-11-27T02:28:10+0900
Minibatch loss at step 1000: 7.942018
Minibatch accu

In [117]:
# Minibatch loss at step 10000: 1.600465
# Minibatch accuracy_length: 96.9%
# Minibatch accuracy_digit_0: 90.6%
# Minibatch accuracy_digit_1: 93.8%
# Minibatch accuracy_digit_2: 96.8%
# Minibatch accuracy_digit_3: 100.0%
# Minibatch accuracy_digit_4: 0.0%
# Minibatch accuracy: 81.2%
# finish : 2016-11-27T03:12:06+0900

# fc1(=64) x 2^3
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=10001, 
                        batch_size=32,  
                        num_convs=[16,32,64,0,0,0,0,0], 
                        num_fc_1=512, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False_False_0.1_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(32, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(32, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(32, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 19.603807
Minibatch accuracy_length: 15.6%
Minibatch accuracy_digit_0: 15.6%
Minibatch accuracy_digit_1: 9.4%
Minibatch accuracy_digit_2: 0.0%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
finish : 2016-11-27T02:44:12+0900
Minibatch loss at step 500: 11.782467
Minibatch accuracy_length: 37.5%
Minibatch accuracy_digit_0: 18.8%
Minibatch accuracy_digit_1: 34.4%
Minibatch accuracy_digit_2: 63.6%
Minibatch accuracy_digit_3: 60.0%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
finish : 2016-11-27T02:45:33+0900
Minibatch loss at step 1000: 9.317673
Minibatch acc

In [118]:
# Minibatch loss at step 10000: 2.042364
# Minibatch accuracy_length: 96.9%
# Minibatch accuracy_digit_0: 96.9%
# Minibatch accuracy_digit_1: 93.8%
# Minibatch accuracy_digit_2: 93.5%
# Minibatch accuracy_digit_3: 100.0%
# Minibatch accuracy_digit_4: 0.0%
# Minibatch accuracy: 81.2%
# finish : 2016-11-27T04:00:55+0900

# fc1(=64) x 2^4
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=10001, 
                        batch_size=32,  
                        num_convs=[16,32,64,0,0,0,0,0], 
                        num_fc_1=1024, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False_False_0.1_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(32, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(32, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(32, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 27.522753
Minibatch accuracy_length: 0.0%
Minibatch accuracy_digit_0: 12.5%
Minibatch accuracy_digit_1: 15.6%
Minibatch accuracy_digit_2: 6.5%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
finish : 2016-11-27T03:12:11+0900
Minibatch loss at step 500: 16.859779
Minibatch accuracy_length: 37.5%
Minibatch accuracy_digit_0: 18.8%
Minibatch accuracy_digit_1: 34.4%
Minibatch accuracy_digit_2: 63.6%
Minibatch accuracy_digit_3: 60.0%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
finish : 2016-11-27T03:14:32+0900
Minibatch loss at step 1000: 13.535250
Minibatch ac

In [119]:
# Minibatch loss at step 10000: 2.811740
# Minibatch accuracy_length: 96.9%
# Minibatch accuracy_digit_0: 90.6%
# Minibatch accuracy_digit_1: 96.9%
# Minibatch accuracy_digit_2: 96.8%
# Minibatch accuracy_digit_3: 100.0%
# Minibatch accuracy_digit_4: 0.0%
# Minibatch accuracy: 84.4%
# finish : 2016-11-27T05:28:28+0900

# fc1(=64) x 2^5
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=10001, 
                        batch_size=32,  
                        num_convs=[16,32,64,0,0,0,0,0], 
                        num_fc_1=2048, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False_False_0.1_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(32, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(32, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(32, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 46.367401
Minibatch accuracy_length: 3.1%
Minibatch accuracy_digit_0: 18.8%
Minibatch accuracy_digit_1: 6.2%
Minibatch accuracy_digit_2: 6.5%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
finish : 2016-11-27T04:01:01+0900
Minibatch loss at step 500: 26.102749
Minibatch accuracy_length: 37.5%
Minibatch accuracy_digit_0: 18.8%
Minibatch accuracy_digit_1: 34.4%
Minibatch accuracy_digit_2: 63.6%
Minibatch accuracy_digit_3: 60.0%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
finish : 2016-11-27T04:05:18+0900
Minibatch loss at step 1000: 22.540548
Minibatch acc

In [120]:
# Minibatch loss at step 100000: 0.557230
# Minibatch accuracy_length: 96.9%
# Minibatch accuracy_digit_0: 96.9%
# Minibatch accuracy_digit_1: 93.8%
# Minibatch accuracy_digit_2: 100.0%
# Minibatch accuracy_digit_3: 100.0%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 90.6%
# finish : 2016-11-27T06:33:55+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=100001, 
                        batch_size=32,  
                        num_convs=[16,32,64,0,0,0,0,0], 
                        num_fc_1=64, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False_False_0.1_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(32, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(32, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(32, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 14.181874
Minibatch accuracy_length: 0.0%
Minibatch accuracy_digit_0: 3.1%
Minibatch accuracy_digit_1: 9.4%
Minibatch accuracy_digit_2: 3.2%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
finish : 2016-11-27T05:28:33+0900
Minibatch loss at step 500: 7.988902
Minibatch accuracy_length: 37.5%
Minibatch accuracy_digit_0: 18.8%
Minibatch accuracy_digit_1: 34.4%
Minibatch accuracy_digit_2: 63.6%
Minibatch accuracy_digit_3: 60.0%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
finish : 2016-11-27T05:28:51+0900
Minibatch loss at step 1000: 6.412196
Minibatch accura

In [160]:
# Minibatch loss at step 100000: 0.601887
# Minibatch accuracy_length: 96.9%
# Minibatch accuracy_digit_0: 96.9%
# Minibatch accuracy_digit_1: 93.8%
# Minibatch accuracy_digit_2: 96.8%
# Minibatch accuracy_digit_3: 94.4%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 90.6%
# Validation accuracy: 86.7%
# finish : 2016-11-28T11:21:13+0900
# Test accuracy: 86.7%
# finish : 2016-11-28T11:21:15+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=100001, 
                        batch_size=32,  
                        num_convs=[16,32,64,0,0,0,0,0], 
                        num_fc_1=64, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False_False_0.1_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(32, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(32, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(32, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 13.938466
Minibatch accuracy_length: 3.1%
Minibatch accuracy_digit_0: 15.6%
Minibatch accuracy_digit_1: 18.8%
Minibatch accuracy_digit_2: 6.5%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
Validation accuracy: 0.6%
finish : 2016-11-28T10:06:57+0900
Minibatch loss at step 500: 8.022201
Minibatch accuracy_length: 37.5%
Minibatch accuracy_digit_0: 18.8%
Minibatch accuracy_digit_1: 34.4%
Minibatch accuracy_digit_2: 63.6%
Minibatch accuracy_digit_3: 60.0%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
Validation accuracy: 0.7%
finish : 2016-11-28T10:07:18+0900


In [161]:
# Minibatch loss at step 100000: 0.657399
# Minibatch accuracy_length: 96.9%
# Minibatch accuracy_digit_0: 100.0%
# Minibatch accuracy_digit_1: 93.8%
# Minibatch accuracy_digit_2: 100.0%
# Minibatch accuracy_digit_3: 100.0%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 93.8%
# Validation accuracy: 87.0%
# finish : 2016-11-28T12:55:27+0900
# Test accuracy: 87.1%
# finish : 2016-11-28T12:55:29+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=100001, 
                        batch_size=32,  
                        num_convs=[16,32,64,64,0,0,0,0], 
                        num_fc_1=64, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False_False_0.1_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(32, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(32, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(32, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 13.478090
Minibatch accuracy_length: 18.8%
Minibatch accuracy_digit_0: 12.5%
Minibatch accuracy_digit_1: 15.6%
Minibatch accuracy_digit_2: 6.5%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
Validation accuracy: 0.6%
finish : 2016-11-28T11:41:31+0900
Minibatch loss at step 500: 8.021799
Minibatch accuracy_length: 37.5%
Minibatch accuracy_digit_0: 18.8%
Minibatch accuracy_digit_1: 34.4%
Minibatch accuracy_digit_2: 63.6%
Minibatch accuracy_digit_3: 60.0%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
Validation accuracy: 0.7%
finish : 2016-11-28T11:41:53+0900

In [162]:
# Minibatch loss at step 100000: 0.413733
# Minibatch accuracy_length: 96.9%
# Minibatch accuracy_digit_0: 100.0%
# Minibatch accuracy_digit_1: 90.6%
# Minibatch accuracy_digit_2: 100.0%
# Minibatch accuracy_digit_3: 100.0%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 90.6%
# Validation accuracy: 89.3%
# finish : 2016-11-28T14:27:39+0900
# Test accuracy: 88.7%
# finish : 2016-11-28T14:27:42+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=100001, 
                        batch_size=32,  
                        num_convs=[16,32,64,64,64,0,0,0], 
                        num_fc_1=64, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False_False_0.1_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(32, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(32, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(32, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 14.323484
Minibatch accuracy_length: 21.9%
Minibatch accuracy_digit_0: 18.8%
Minibatch accuracy_digit_1: 6.2%
Minibatch accuracy_digit_2: 25.8%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
Validation accuracy: 0.5%
finish : 2016-11-28T13:12:03+0900
Minibatch loss at step 500: 8.015701
Minibatch accuracy_length: 37.5%
Minibatch accuracy_digit_0: 18.8%
Minibatch accuracy_digit_1: 34.4%
Minibatch accuracy_digit_2: 63.6%
Minibatch accuracy_digit_3: 60.0%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
Validation accuracy: 0.7%
finish : 2016-11-28T13:12:25+0900

In [163]:
# Minibatch loss at step 100000: 0.363833
# Minibatch accuracy_length: 96.9%
# Minibatch accuracy_digit_0: 100.0%
# Minibatch accuracy_digit_1: 93.8%
# Minibatch accuracy_digit_2: 100.0%
# Minibatch accuracy_digit_3: 100.0%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 93.8%
# Validation accuracy: 89.5%
# finish : 2016-11-28T15:45:28+0900
# Test accuracy: 89.7%
# finish : 2016-11-28T15:45:31+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=100001, 
                        batch_size=32,  
                        num_convs=[16,32,64,64,64,64,0,0], 
                        num_fc_1=64, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False_False_0.1_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(32, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(32, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(32, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 14.073882
Minibatch accuracy_length: 3.1%
Minibatch accuracy_digit_0: 9.4%
Minibatch accuracy_digit_1: 31.2%
Minibatch accuracy_digit_2: 6.5%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
Validation accuracy: 0.6%
finish : 2016-11-28T14:27:53+0900
Minibatch loss at step 500: 8.007686
Minibatch accuracy_length: 37.5%
Minibatch accuracy_digit_0: 18.8%
Minibatch accuracy_digit_1: 34.4%
Minibatch accuracy_digit_2: 63.6%
Minibatch accuracy_digit_3: 60.0%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
Validation accuracy: 0.7%
finish : 2016-11-28T14:28:15+0900
M

In [164]:
# Minibatch loss at step 100000: 0.323064
# Minibatch accuracy_length: 96.9%
# Minibatch accuracy_digit_0: 100.0%
# Minibatch accuracy_digit_1: 96.9%
# Minibatch accuracy_digit_2: 100.0%
# Minibatch accuracy_digit_3: 100.0%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 96.9%
# Validation accuracy: 89.9%
# finish : 2016-11-28T17:10:10+0900
# Test accuracy: 89.8%
# finish : 2016-11-28T17:10:13+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=100001, 
                        batch_size=32,  
                        num_convs=[16,32,64,64,64,64,64,0], 
                        num_fc_1=64, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False_False_0.1_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(32, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(32, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(32, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 14.700812
Minibatch accuracy_length: 3.1%
Minibatch accuracy_digit_0: 21.9%
Minibatch accuracy_digit_1: 9.4%
Minibatch accuracy_digit_2: 19.4%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 3.1%
Validation accuracy: 0.6%
finish : 2016-11-28T15:45:42+0900
Minibatch loss at step 500: 8.015172
Minibatch accuracy_length: 37.5%
Minibatch accuracy_digit_0: 18.8%
Minibatch accuracy_digit_1: 34.4%
Minibatch accuracy_digit_2: 63.6%
Minibatch accuracy_digit_3: 60.0%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
Validation accuracy: 0.7%
finish : 2016-11-28T15:46:07+0900


In [165]:
# Minibatch loss at step 100000: 0.160149
# Minibatch accuracy_length: 100.0%
# Minibatch accuracy_digit_0: 100.0%
# Minibatch accuracy_digit_1: 100.0%
# Minibatch accuracy_digit_2: 100.0%
# Minibatch accuracy_digit_3: 100.0%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 100.0%
# Validation accuracy: 90.1%
# finish : 2016-11-28T18:39:22+0900
# Test accuracy: 89.6%
# finish : 2016-11-28T18:39:25+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=100001, 
                        batch_size=32,  
                        num_convs=[16,32,64,64,64,64,64,64], 
                        num_fc_1=64, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False_False_0.1_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(32, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(32, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(32, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 14.772124
Minibatch accuracy_length: 0.0%
Minibatch accuracy_digit_0: 15.6%
Minibatch accuracy_digit_1: 3.1%
Minibatch accuracy_digit_2: 0.0%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
Validation accuracy: 0.6%
finish : 2016-11-28T17:10:24+0900
Minibatch loss at step 500: 8.031597
Minibatch accuracy_length: 37.5%
Minibatch accuracy_digit_0: 18.8%
Minibatch accuracy_digit_1: 34.4%
Minibatch accuracy_digit_2: 63.6%
Minibatch accuracy_digit_3: 60.0%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
Validation accuracy: 0.7%
finish : 2016-11-28T17:10:49+0900
M

In [166]:
# Minibatch loss at step 100000: 0.270255
# Minibatch accuracy_length: 96.9%
# Minibatch accuracy_digit_0: 100.0%
# Minibatch accuracy_digit_1: 96.9%
# Minibatch accuracy_digit_2: 100.0%
# Minibatch accuracy_digit_3: 100.0%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 96.9%
# Validation accuracy: 90.0%
# finish : 2016-11-28T20:41:24+0900
# Test accuracy: 89.5%
# finish : 2016-11-28T20:41:27+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=100001, 
                        batch_size=32,  
                        num_convs=[16,32,64,64,64,64,64,64], 
                        num_fc_1=128, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False_False_0.1_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(32, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(32, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(32, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 14.113474
Minibatch accuracy_length: 34.4%
Minibatch accuracy_digit_0: 12.5%
Minibatch accuracy_digit_1: 0.0%
Minibatch accuracy_digit_2: 3.2%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
Validation accuracy: 0.3%
finish : 2016-11-28T18:39:37+0900
Minibatch loss at step 500: 8.592757
Minibatch accuracy_length: 37.5%
Minibatch accuracy_digit_0: 18.8%
Minibatch accuracy_digit_1: 34.4%
Minibatch accuracy_digit_2: 63.6%
Minibatch accuracy_digit_3: 60.0%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
Validation accuracy: 0.7%
finish : 2016-11-28T18:40:12+0900


In [167]:
# Minibatch loss at step 100000: 0.648583
# Minibatch accuracy_length: 96.9%
# Minibatch accuracy_digit_0: 100.0%
# Minibatch accuracy_digit_1: 90.6%
# Minibatch accuracy_digit_2: 100.0%
# Minibatch accuracy_digit_3: 100.0%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 90.6%
# Validation accuracy: 90.6%
# finish : 2016-11-29T00:08:29+0900
# Test accuracy: 90.1%
# finish : 2016-11-29T00:08:33+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=100001, 
                        batch_size=32,  
                        num_convs=[16,32,64,64,64,64,64,128], 
                        num_fc_1=128, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False_False_0.1_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(32, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(32, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(32, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 16.426882
Minibatch accuracy_length: 75.0%
Minibatch accuracy_digit_0: 0.0%
Minibatch accuracy_digit_1: 6.2%
Minibatch accuracy_digit_2: 3.2%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
Validation accuracy: 0.6%
finish : 2016-11-28T20:41:39+0900
Minibatch loss at step 500: 9.739598
Minibatch accuracy_length: 37.5%
Minibatch accuracy_digit_0: 18.8%
Minibatch accuracy_digit_1: 34.4%
Minibatch accuracy_digit_2: 63.6%
Minibatch accuracy_digit_3: 60.0%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
Validation accuracy: 0.7%
finish : 2016-11-28T20:42:37+0900
M

In [168]:
# Minibatch loss at step 100000: 0.137252
# Minibatch accuracy_length: 100.0%
# Minibatch accuracy_digit_0: 100.0%
# Minibatch accuracy_digit_1: 100.0%
# Minibatch accuracy_digit_2: 100.0%
# Minibatch accuracy_digit_3: 100.0%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 100.0%
# Validation accuracy: 89.9%
# finish : 2016-11-29T03:36:36+0900
# Test accuracy: 89.7%
# finish : 2016-11-29T03:36:40+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=100001, 
                        batch_size=32,  
                        num_convs=[16,32,64,64,64,64,128,128], 
                        num_fc_1=128, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False_False_0.1_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(32, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(32, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(32, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 18.937731
Minibatch accuracy_length: 3.1%
Minibatch accuracy_digit_0: 9.4%
Minibatch accuracy_digit_1: 31.2%
Minibatch accuracy_digit_2: 0.0%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
Validation accuracy: 0.3%
finish : 2016-11-29T00:08:46+0900
Minibatch loss at step 500: 9.751569
Minibatch accuracy_length: 37.5%
Minibatch accuracy_digit_0: 18.8%
Minibatch accuracy_digit_1: 34.4%
Minibatch accuracy_digit_2: 63.6%
Minibatch accuracy_digit_3: 60.0%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
Validation accuracy: 0.7%
finish : 2016-11-29T00:09:43+0900
M

In [176]:
# Minibatch loss at step 200000: 0.061928
# Minibatch accuracy_length: 100.0%
# Minibatch accuracy_digit_0: 100.0%
# Minibatch accuracy_digit_1: 100.0%
# Minibatch accuracy_digit_2: 100.0%
# Minibatch accuracy_digit_3: 100.0%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 100.0%
# Validation accuracy: 89.6%
# finish : 2016-11-29T11:34:08+0900
# Test accuracy: 89.1%
# finish : 2016-11-29T11:34:12+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=200001, 
                        batch_size=32,  
                        num_convs=[16,32,64,64,64,64,64,64], 
                        num_fc_1=64, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False_False_0.1_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(32, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(32, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(32, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 13.509753
Minibatch accuracy_length: 75.0%
Minibatch accuracy_digit_0: 6.2%
Minibatch accuracy_digit_1: 6.2%
Minibatch accuracy_digit_2: 3.2%
Minibatch accuracy_digit_3: 100.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
Validation accuracy: 0.6%
finish : 2016-11-29T08:45:21+0900
Minibatch loss at step 500: 8.016180
Minibatch accuracy_length: 37.5%
Minibatch accuracy_digit_0: 18.8%
Minibatch accuracy_digit_1: 34.4%
Minibatch accuracy_digit_2: 63.6%
Minibatch accuracy_digit_3: 60.0%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
Validation accuracy: 0.7%
finish : 2016-11-29T08:45:45+0900

In [177]:
# Minibatch loss at step 100000: 0.076815
# Minibatch accuracy_length: 100.0%
# Minibatch accuracy_digit_0: 100.0%
# Minibatch accuracy_digit_1: 100.0%
# Minibatch accuracy_digit_2: 100.0%
# Minibatch accuracy_digit_3: 100.0%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 100.0%
# Validation accuracy: 89.4%
# finish : 2016-11-29T15:14:29+0900
# Test accuracy: 89.1%
# finish : 2016-11-29T15:14:31+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=100001, 
                        batch_size=64,  
                        num_convs=[16,32,64,64,64,64,64,64], 
                        num_fc_1=64, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False_False_0.1_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(64, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(64, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(64, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 13.844358
Minibatch accuracy_length: 20.3%
Minibatch accuracy_digit_0: 4.7%
Minibatch accuracy_digit_1: 9.4%
Minibatch accuracy_digit_2: 51.7%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
Validation accuracy: 0.5%
finish : 2016-11-29T13:44:23+0900
Minibatch loss at step 500: 7.050818
Minibatch accuracy_length: 59.4%
Minibatch accuracy_digit_0: 29.7%
Minibatch accuracy_digit_1: 32.8%
Minibatch accuracy_digit_2: 75.0%
Minibatch accuracy_digit_3: 85.7%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 3.1%
Validation accuracy: 0.7%
finish : 2016-11-29T13:44:48+0900


In [178]:
# Minibatch loss at step 100000: 0.086014
# Minibatch accuracy_length: 100.0%
# Minibatch accuracy_digit_0: 100.0%
# Minibatch accuracy_digit_1: 100.0%
# Minibatch accuracy_digit_2: 100.0%
# Minibatch accuracy_digit_3: 100.0%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 100.0%
# Validation accuracy: 89.6%
# finish : 2016-11-29T18:27:38+0900
# Test accuracy: 89.5%
# finish : 2016-11-29T18:27:41+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=100001, 
                        batch_size=64,  
                        num_convs=[32,32,32,64,64,64,64,64], 
                        num_fc_1=64, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False_False_0.1_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(64, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(64, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(64, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 13.715680
Minibatch accuracy_length: 71.9%
Minibatch accuracy_digit_0: 3.1%
Minibatch accuracy_digit_1: 12.5%
Minibatch accuracy_digit_2: 11.7%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
Validation accuracy: 0.6%
finish : 2016-11-29T15:43:03+0900
Minibatch loss at step 500: 7.048334
Minibatch accuracy_length: 59.4%
Minibatch accuracy_digit_0: 29.7%
Minibatch accuracy_digit_1: 32.8%
Minibatch accuracy_digit_2: 75.0%
Minibatch accuracy_digit_3: 85.7%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 3.1%
Validation accuracy: 0.7%
finish : 2016-11-29T15:43:59+0900

In [179]:
# Minibatch loss at step 100000: 0.432907
# Minibatch accuracy_length: 96.9%
# Minibatch accuracy_digit_0: 100.0%
# Minibatch accuracy_digit_1: 93.8%
# Minibatch accuracy_digit_2: 100.0%
# Minibatch accuracy_digit_3: 100.0%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 93.8%
# Validation accuracy: 90.2%
# finish : 2016-11-30T00:33:23+0900
# Test accuracy: 89.8%
# finish : 2016-11-30T00:33:27+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=100001, 
                        batch_size=32,  
                        num_convs=[16,32,64,64,64,64,128,128], 
                        num_fc_1=128, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False_False_0.1_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(32, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(32, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(32, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 19.339659
Minibatch accuracy_length: 3.1%
Minibatch accuracy_digit_0: 9.4%
Minibatch accuracy_digit_1: 9.4%
Minibatch accuracy_digit_2: 3.2%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
Validation accuracy: 0.2%
finish : 2016-11-29T21:10:51+0900
Minibatch loss at step 500: 9.754180
Minibatch accuracy_length: 37.5%
Minibatch accuracy_digit_0: 18.8%
Minibatch accuracy_digit_1: 34.4%
Minibatch accuracy_digit_2: 63.6%
Minibatch accuracy_digit_3: 60.0%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
Validation accuracy: 0.7%
finish : 2016-11-29T21:11:48+0900
Mi

In [180]:
# Minibatch loss at step 100000: 0.437522
# Minibatch accuracy_length: 96.9%
# Minibatch accuracy_digit_0: 96.9%
# Minibatch accuracy_digit_1: 96.9%
# Minibatch accuracy_digit_2: 100.0%
# Minibatch accuracy_digit_3: 100.0%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 93.8%
# Validation accuracy: 89.3%
# finish : 2016-11-30T03:52:08+0900
# Test accuracy: 89.0%
# finish : 2016-11-30T03:52:12+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=100001, 
                        batch_size=32,  
                        num_convs=[16,32,64,64,64,128,128,128], 
                        num_fc_1=128, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False_False_0.1_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(32, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(32, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(32, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 18.294018
Minibatch accuracy_length: 46.9%
Minibatch accuracy_digit_0: 6.2%
Minibatch accuracy_digit_1: 15.6%
Minibatch accuracy_digit_2: 0.0%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
Validation accuracy: 0.6%
finish : 2016-11-30T00:33:39+0900
Minibatch loss at step 500: 9.840426
Minibatch accuracy_length: 37.5%
Minibatch accuracy_digit_0: 18.8%
Minibatch accuracy_digit_1: 34.4%
Minibatch accuracy_digit_2: 63.6%
Minibatch accuracy_digit_3: 60.0%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
Validation accuracy: 0.7%
finish : 2016-11-30T00:34:37+0900


In [181]:
# Minibatch loss at step 100000: 0.258780
# Minibatch accuracy_length: 96.9%
# Minibatch accuracy_digit_0: 100.0%
# Minibatch accuracy_digit_1: 93.8%
# Minibatch accuracy_digit_2: 100.0%
# Minibatch accuracy_digit_3: 100.0%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 93.8%
# Validation accuracy: 87.6%
# finish : 2016-11-30T07:10:31+0900
# Test accuracy: 87.7%
# finish : 2016-11-30T07:10:36+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=100001, 
                        batch_size=32,  
                        num_convs=[16,32,64,64,128,128,128,128], 
                        num_fc_1=128, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False_False_0.1_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("Placeholder:0", shape=(32, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder_1:0", shape=(32, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_2:0", shape=(32, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 21.581238
Minibatch accuracy_length: 0.0%
Minibatch accuracy_digit_0: 12.5%
Minibatch accuracy_digit_1: 3.1%
Minibatch accuracy_digit_2: 6.5%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
Validation accuracy: 0.6%
finish : 2016-11-30T03:52:24+0900
Minibatch loss at step 500: 9.767053
Minibatch accuracy_length: 37.5%
Minibatch accuracy_digit_0: 18.8%
Minibatch accuracy_digit_1: 34.4%
Minibatch accuracy_digit_2: 63.6%
Minibatch accuracy_digit_3: 60.0%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
Validation accuracy: 0.7%
finish : 2016-11-30T03:53:21+0900
M

In [190]:
# Minibatch loss at step 100000: 0.481327
# Minibatch accuracy_length: 96.9%
# Minibatch accuracy_digit_0: 96.9%
# Minibatch accuracy_digit_1: 93.8%
# Minibatch accuracy_digit_2: 96.8%
# Minibatch accuracy_digit_3: 100.0%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 87.5%
# Validation accuracy: 88.3%
# finish : 2016-11-30T12:54:39+0900
# Test accuracy: 88.3%
# finish : 2016-11-30T12:54:44+0900
# Converted 32 variables to const ops.
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        use_drop_out=True,
                        drop_out_rate=0.9,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=100001, 
                        batch_size=32,  
                        num_convs=[16,32,64,64,64,64,128,128], 
                        num_fc_1=128, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False_False_0.1_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("tf_train_dataset:0", shape=(32, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder:0", shape=(32, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_1:0", shape=(32, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 16.282745
Minibatch accuracy_length: 75.0%
Minibatch accuracy_digit_0: 15.6%
Minibatch accuracy_digit_1: 3.1%
Minibatch accuracy_digit_2: 0.0%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
Validation accuracy: 0.6%
finish : 2016-11-30T09:36:40+0900
Minibatch loss at step 500: 9.727321
Minibatch accuracy_length: 37.5%
Minibatch accuracy_digit_0: 18.8%
Minibatch accuracy_digit_1: 34.4%
Minibatch accuracy_digit_2: 63.6%
Minibatch accuracy_digit_3: 60.0%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
Validation accuracy: 0.7%
finish : 2016-11-30T09:37:36+09

In [191]:
# Minibatch loss at step 100000: 0.341554
# Minibatch accuracy_length: 96.9%
# Minibatch accuracy_digit_0: 100.0%
# Minibatch accuracy_digit_1: 96.9%
# Minibatch accuracy_digit_2: 100.0%
# Minibatch accuracy_digit_3: 100.0%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 96.9%
# Validation accuracy: 89.2%
# finish : 2016-11-30T16:18:30+0900
# Test accuracy: 89.3%
# finish : 2016-11-30T16:18:34+0900
# Converted 32 variables to const ops.
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        use_drop_out=True,
                        drop_out_rate=0.8,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=100001, 
                        batch_size=32,  
                        num_convs=[16,32,64,64,64,64,128,128], 
                        num_fc_1=128, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False_False_0.1_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("tf_train_dataset:0", shape=(32, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder:0", shape=(32, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_1:0", shape=(32, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 17.235853
Minibatch accuracy_length: 25.0%
Minibatch accuracy_digit_0: 3.1%
Minibatch accuracy_digit_1: 12.5%
Minibatch accuracy_digit_2: 3.2%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
Validation accuracy: 0.6%
finish : 2016-11-30T12:54:58+0900
Minibatch loss at step 500: 9.753815
Minibatch accuracy_length: 37.5%
Minibatch accuracy_digit_0: 18.8%
Minibatch accuracy_digit_1: 34.4%
Minibatch accuracy_digit_2: 63.6%
Minibatch accuracy_digit_3: 60.0%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
Validation accuracy: 0.7%
finish : 2016-11-30T12:55:55+09

In [192]:
# Minibatch loss at step 100000: 0.364618
# Minibatch accuracy_length: 96.9%
# Minibatch accuracy_digit_0: 96.9%
# Minibatch accuracy_digit_1: 90.6%
# Minibatch accuracy_digit_2: 100.0%
# Minibatch accuracy_digit_3: 100.0%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 87.5%
# Validation accuracy: 89.3%
# finish : 2016-11-30T19:40:37+0900
# Test accuracy: 89.2%
# finish : 2016-11-30T19:40:41+0900
# Converted 32 variables to const ops.
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        use_drop_out=True,
                        drop_out_rate=0.7,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=100001, 
                        batch_size=32,  
                        num_convs=[16,32,64,64,64,64,128,128], 
                        num_fc_1=128, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False_False_0.1_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("tf_train_dataset:0", shape=(32, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder:0", shape=(32, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_1:0", shape=(32, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 16.286360
Minibatch accuracy_length: 0.0%
Minibatch accuracy_digit_0: 15.6%
Minibatch accuracy_digit_1: 6.2%
Minibatch accuracy_digit_2: 0.0%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
Validation accuracy: 0.6%
finish : 2016-11-30T16:18:48+0900
Minibatch loss at step 500: 9.789307
Minibatch accuracy_length: 37.5%
Minibatch accuracy_digit_0: 18.8%
Minibatch accuracy_digit_1: 34.4%
Minibatch accuracy_digit_2: 63.6%
Minibatch accuracy_digit_3: 60.0%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
Validation accuracy: 0.7%
finish : 2016-11-30T16:19:44+090

In [193]:
# Minibatch loss at step 100000: 0.661321
# Minibatch accuracy_length: 96.9%
# Minibatch accuracy_digit_0: 100.0%
# Minibatch accuracy_digit_1: 93.8%
# Minibatch accuracy_digit_2: 100.0%
# Minibatch accuracy_digit_3: 100.0%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 93.8%
# Validation accuracy: 89.9%
# finish : 2016-11-30T22:59:59+0900
# Test accuracy: 90.1%
# finish : 2016-11-30T23:00:03+0900
# Converted 32 variables to const ops.
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        use_drop_out=True,
                        drop_out_rate=0.6,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=100001, 
                        batch_size=32,  
                        num_convs=[16,32,64,64,64,64,128,128], 
                        num_fc_1=128, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False_False_0.1_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("tf_train_dataset:0", shape=(32, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder:0", shape=(32, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_1:0", shape=(32, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 19.699005
Minibatch accuracy_length: 12.5%
Minibatch accuracy_digit_0: 0.0%
Minibatch accuracy_digit_1: 6.2%
Minibatch accuracy_digit_2: 0.0%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
Validation accuracy: 0.5%
finish : 2016-11-30T19:40:53+0900
Minibatch loss at step 500: 9.707457
Minibatch accuracy_length: 37.5%
Minibatch accuracy_digit_0: 18.8%
Minibatch accuracy_digit_1: 34.4%
Minibatch accuracy_digit_2: 63.6%
Minibatch accuracy_digit_3: 60.0%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
Validation accuracy: 0.7%
finish : 2016-11-30T19:41:50+090

In [194]:
# Minibatch loss at step 100000: 0.269569
# Minibatch accuracy_length: 96.9%
# Minibatch accuracy_digit_0: 100.0%
# Minibatch accuracy_digit_1: 96.9%
# Minibatch accuracy_digit_2: 100.0%
# Minibatch accuracy_digit_3: 100.0%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 96.9%
# Validation accuracy: 89.4%
# finish : 2016-12-01T02:18:29+0900
# Test accuracy: 89.4%
# finish : 2016-12-01T02:18:33+0900
# Converted 32 variables to const ops.
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        use_drop_out=True,
                        drop_out_rate=0.5,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=100001, 
                        batch_size=32,  
                        num_convs=[16,32,64,64,64,64,128,128], 
                        num_fc_1=128, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False_False_0.1_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("tf_train_dataset:0", shape=(32, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder:0", shape=(32, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_1:0", shape=(32, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 19.066032
Minibatch accuracy_length: 3.1%
Minibatch accuracy_digit_0: 18.8%
Minibatch accuracy_digit_1: 0.0%
Minibatch accuracy_digit_2: 9.7%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
Validation accuracy: 0.5%
finish : 2016-11-30T23:00:16+0900
Minibatch loss at step 500: 9.819472
Minibatch accuracy_length: 37.5%
Minibatch accuracy_digit_0: 18.8%
Minibatch accuracy_digit_1: 34.4%
Minibatch accuracy_digit_2: 63.6%
Minibatch accuracy_digit_3: 60.0%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
Validation accuracy: 0.7%
finish : 2016-11-30T23:01:12+090

In [ ]:
# Minibatch loss at step 100000: 0.685685
# Minibatch accuracy_length: 96.9%
# Minibatch accuracy_digit_0: 96.9%
# Minibatch accuracy_digit_1: 93.8%
# Minibatch accuracy_digit_2: 100.0%
# Minibatch accuracy_digit_3: 100.0%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 90.6%
# Validation accuracy: 86.9%
# finish : 2016-12-01T06:06:45+0900
# Test accuracy: 86.7%
# finish : 2016-12-01T06:06:49+0900
# Converted 32 variables to const ops.
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32, add_margin=True, margin_ratio=0.1)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        use_drop_out=True,
                        drop_out_rate=0.6,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=100001, 
                        batch_size=32,  
                        num_convs=[16,32,64,64,64,64,128,128], 
                        num_fc_1=128, 
                        num_fc_2=64,
                       ).run()

Pickling svhn/reader/svhn_5_32_300000_False_True_0.1_False.pickle.
('rows[5][0]', (3, 350, 741), dtype('uint8'))
('total_data', (248822, 32, 32, 3))
('total_label', (248822, 6))
start : 2016-12-01T02:21:22+0900
end : 2016-12-01T02:45:23+0900
('train_data', (223939, 32, 32, 3))
('train_label', (223939, 6))
('validation_data', (12441, 32, 32, 3))
('validation_label', (12441, 6))
('test_data', (12442, 32, 32, 3))
('test_label', (12442, 6))
tf_train_dataset : Tensor("tf_train_dataset:0", shape=(32, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder:0", shape=(32, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_1:0", shape=(32, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 18.001881
Minibatch accuracy_length: 3.1%
Minibatch accuracy_digit_0: 12.5%
Minibatch accuracy_digit_1: 9.4%
Minibatch accuracy_digit_2: 12.9%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
Validation accuracy: 0.7%
finish : 20

In [ ]:
# Minibatch loss at step 49500: 0.980998
# Minibatch accuracy_length: 96.9%
# Minibatch accuracy_digit_0: 100.0%
# Minibatch accuracy_digit_1: 87.5%
# Minibatch accuracy_digit_2: 88.5%
# Minibatch accuracy_digit_3: 100.0%
# Minibatch accuracy_digit_4: 100.0%
# Minibatch accuracy: 78.1%
# Validation accuracy: 87.9%
# finish : 2016-12-01T09:32:23+0900
svhn_data = MultiDigitSVHNData.maybe_pickle(digit_count=5, total_data_count=300000, image_size=32)
MultiDigitModel(multi_digit_dataset=svhn_data,
                        use_norm=True,
                        learning_rate_start=0.01,
                        digit_count=5,
                        num_channels=3,
                        num_steps=200001, 
                        batch_size=32,  
                        num_convs=[16,32,64,64,64,64,64,128], 
                        num_fc_1=128, 
                        num_fc_2=64,
                       ).run()

svhn/reader/svhn_5_32_300000_False_False_0.1_False.pickle already present - Skipping pickling.
tf_train_dataset : Tensor("tf_train_dataset:0", shape=(32, 32, 32, 3), dtype=float32)
tf_train_length_labels : Tensor("Placeholder:0", shape=(32, 6), dtype=float32)
tf_train_digits_labels : Tensor("Placeholder_1:0", shape=(32, 5, 10), dtype=float32)
Initialized
Minibatch loss at step 0: 16.951754
Minibatch accuracy_length: 3.1%
Minibatch accuracy_digit_0: 18.8%
Minibatch accuracy_digit_1: 12.5%
Minibatch accuracy_digit_2: 6.5%
Minibatch accuracy_digit_3: 0.0%
Minibatch accuracy_digit_4: 0.0%
Minibatch accuracy: 0.0%
Validation accuracy: 0.6%
finish : 2016-12-01T07:53:10+0900
Minibatch loss at step 500: 9.746739
Minibatch accuracy_length: 37.5%
Minibatch accuracy_digit_0: 18.8%
Minibatch accuracy_digit_1: 34.4%
Minibatch accuracy_digit_2: 63.6%
Minibatch accuracy_digit_3: 60.0%
Minibatch accuracy_digit_4: 100.0%
Minibatch accuracy: 0.0%
Validation accuracy: 0.7%
finish : 2016-12-01T07:54:06+09